In [1]:
# to do:
# Make function for bar plot and add legend - light and dark phase
# address the problem of mean calculation of data_summary function in plotting per gene plot
# add time point title for circular plot

### Load the packages

In [2]:
inst <- suppressMessages(lapply(c("edgeR",
                                  "MetaCycle",
                                  "lubridate",
                                  "plyr",
                                  "dplyr",
                                  "tidyr",
                                  "RColorBrewer",
                                  "ggplot2",
                                  "gplots",
                                  "grid",
                                  "gridExtra",
                                  "DESeq2",
                                  "VennDiagram",
                                  "org.Rn.eg.db",
                                  "clusterProfiler",
                                  "pheatmap",
                                  "ShrinkBayes",
                                  "Hmisc",
                                  "circlize",
                                  "lineup",
                                  "globaltest",
                                  "readxl",
                                  "multipanelfigure",
                                  "UpSetR",
                                  "ggrepel",
                                  "huge",
                                  "ragt2ridges"
                                ),
                                library,
                                character.only = TRUE)
) 
# set the directory
dir_out = '~/Documents/consultation/ChunXia/Rat_timedata/circadian_rat/results'
dir_tab = file.path(dir_out, 'tables')
dir_fig = file.path(dir_out, 'figures')
dir_dat = file.path(dir_out, 'anndata')

# set the colors and options
colors <- colorRampPalette(rev(brewer.pal(9, "Spectral")))(n=255)
my_palette <- colorRampPalette(c("blue", "white", "red"))(n=255)
options(repr.plot.width = 10, repr.plot.height = 10)

### Set the global variables

Set whether anndata objects are recomputed or loaded from cache.

In [3]:
bool_recomp = FALSE

Set whether to produce plots, set to False for test runs.

In [4]:
bool_plot = FALSE

### Load data and functions

In [5]:
# set input file
setwd('~/Documents/consultation/ChunXia/Rat_timedata/circadian_rat/input')
# load the data
dat = read.csv('circadian_data.csv', row.names = 1)
pfdat <- read.csv("additional_data.txt", sep='')

# load the functions
source('circadian_functions.R')
source('tigaR_functions.R')

# define the conditions
id <- colnames(dat)
tissue <- c(rep("hippo", 60), rep("hpt", 60))
timepoint <- as.factor(rep(c(rep(2, 5), rep(6, 5), rep(10, 5), rep(14, 5), rep(18, 5), rep(22, 5)), 4))
tigar_tp <- rep(1:6,20)
diet <- c(rep("chow", 30), rep("hfd", 30), rep("chow", 30), rep("hfd", 30))
condition <- paste(tissue, timepoint, diet, sep = '')
tissue_diet <- paste(tissue, diet, sep = '_')

# load the target genes per group
if(bool_recomp){
    groups = c('clock', 'obesity', 'diabetes', 'synapticPlastisty', 'insulinPath', 'cytokinRecpt',
               'fattyAcidMetab', 'glucoseMetab', 'tightJunction', 'endothelia', 'Pathways', 'KEGG', 'hpt_hip_genes')

    targetGenes <- list()
    for(i in 1:length(groups)){
        targetGenes[[groups[i]]] <- pull(read_excel('target_genes.xlsx', sheet = i)[,3])
    }
    saveRDS(targetGenes, file.path(dir_dat, 'target_genes.RDS'))
} else{
    targetGenes = readRDS(file.path(dir_dat, 'target_genes.RDS'))
}

### Filtering

In [6]:
# Remove problematics samples and make a metadata for DESeq object
prob_samp <- c("hippoTP2hfd.2", "hptTP2hfd.2", "hippoTP14chow.2", "hptTP14chow.2", "hippoTP18chow.2",
               "hptTP18chow.2", "hippoTP22chow.2", "hptTP22chow.2", "hptTP18hfd", "hippoTP6chow", "hippoTP6chow.2") 

if(bool_recomp){
    tissue_tp <- paste(tissue,timepoint, sep = '')
    tp_sum <- row_sum <- numeric()
    for(i in unique(tissue_tp)){
        tp_sum <- cbind(tp_sum,rowSums(dat[,tissue_tp == i])) # sum of the counts per timepoint both for chow and hfd
        row_sum <- cbind(row_sum,rowSums(dat[,tissue_tp == i] != 0)) # count the number of sample with count >0
    }
    #Keep genes with more then 3 samples with >0 number of counts per each timepoint(both chow and hfd)
    keep <- rowSums(row_sum < 4) < 2 
    dat <- dat[keep,]
    tp_sum <- tp_sum[keep,]

    # Keep the genes which have sum of the counts per timepoint(both chow and hfd) higher then 100
    dat_filt <- dat[rowSums(tp_sum >= 100) >= 12,]

    # Remove redundant genes
    #dat_filt <- dat_filt[-(grep("^(Fam|Tmem|NEWGENE|MGC|RT1|RGD|AC|LOC|AABR|mt|Rp(l|s)|Gm\\d)",
    #                            rownames(dat_filt))),]  
    
    dat_filt <- dat_filt[-(grep("^(AC|LOC|NEWGENE_6|AABR\\d)",
                                rownames(dat_filt))),]  
    
    # Remove problematic samples
    dat_filt <- dat_filt[,!colnames(dat_filt) %in% prob_samp]
    # Save the filtered raw data 
    write.csv(dat_filt, file.path(dir_dat, 'raw_filt_data.csv'))
} else{
    dat_filt = read.csv(file.path(dir_dat, 'raw_filt_data.csv'), 
                        row.names = 1
    )
}
dim(dat_filt)

[1] 13432   109

### DESeq2 object

In [7]:
# conditions
id <- id[!colnames(dat) %in% prob_samp]
tissue <- tissue[!colnames(dat) %in% prob_samp]
timepoint <- timepoint[!colnames(dat) %in% prob_samp]
tigar_tp <- tigar_tp[!colnames(dat) %in% prob_samp]
diet <- diet[!colnames(dat) %in% prob_samp]
condition <- condition[!colnames(dat) %in% prob_samp]
tissue_diet <- tissue_diet[!colnames(dat) %in% prob_samp]
# meta data
metaData <- data.frame(id, tissue, timepoint, diet, condition)

# DESeq object
dds <- DESeqDataSetFromMatrix(countData = dat_filt,
                              colData = metaData,
                              design =~ timepoint + diet + tissue
)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


In [8]:
# normalized data
if(bool_recomp){
    # estimate size factors
    dds <- estimateSizeFactors(dds)
    dat_norm <- round(counts(dds, normalized = TRUE), 0)
    # save the normalized data
    write.csv(dat_norm, 
              file.path(dir_dat, "normalized_data.csv"),
              row.names = T
    )
} else{
    dat_norm = read.csv(file.path(dir_dat, 'raw_filt_data.csv'), 
                        row.names = 1
    )
}

In [9]:
# variance stabilizing transformation data
vsd <- assay(vst(dds, 
                 fitType="local",
                 blind = TRUE
             )
        )

### Visualize samples

In [10]:
# PCA plot
if(bool_plot){
    pcaData <- plotPCA(vst(dds, fitType = "local"),
                       intgroup = c("condition"),
                       returnData = TRUE
                )
    percentVar <- round(100 * attr(pcaData, "percentVar"))
    pca <- ggplot(pcaData, aes(PC1, PC2, color = condition)) +
                  geom_point(size = 3) +
                  geom_label(aes(label = colnames(dat_filt))) +
                  xlab(paste0("PC1: ", percentVar[1], "% variance")) + 
                  ylab(paste0("PC2: ", percentVar[2], "% variance")) +
                  guides(color = guide_legend(ncol = 1)) +
                  ggtitle("PCA Plot")
    
    ggsave(file.path(dir_fig, "pca_plot.pdf"))
    print(pca)
}

In [11]:
# sample distances matrix heatmap
if(bool_plot){
    sampleDistMatrix <- as.matrix(dist(t(vsd)))

    # Data frame with column annotations.
    col_ann <- as.data.frame(colData(dds)[, c('diet', 'timepoint', 'tissue')])
    rownames(col_ann) <- colnames(sampleDistMatrix)

    # List with colors for each annotation.
    ann_colors <- list(condition = brewer.pal(12, "Set3"))
    names(ann_colors$condition) <- unique(colData(dds)$timepoint)
    
    sdm <- pheatmap(sampleDistMatrix,
                    clustering_distance_rows = dist(t(vsd)),
                    clustering_distance_cols = dist(t(vsd)),
                    show_colnames = TRUE,
                    show_rownames = FALSE,
                    annotation = col_ann,
                    annotation_colors = ann_colors,
                    col = colors
    )
    save_pheatmap_pdf(sdm,
                      file.path(dir_fig, "sample_distance_heatmap.pdf"))
}

In [12]:
# Heatmpa of all samples and genes
if(bool_plot){  
    agsh <- pheatmap(vsd,
                     cluster_rows = T, 
                     scale = "row", 
                     show_rownames = FALSE, 
                     color = my_palette,
                     cluster_cols = T, 
                     annotation_col = as.data.frame(colData(dds)[,c("diet", "timepoint", "tissue")]),
                     fontsize = 8,
                     breaks = seq(-5, 5, length.out = 255)
    )
    save_pheatmap_pdf(agsh,
                      file.path(dir_fig, "all_genes_samples_heatmap.pdf"))
}

## Circadian gene expression analysis

In [13]:
# write into .csv files data for JTK
if(bool_recomp){
    # dat_norm - counts normalised data
    # vsd - continious normalised data
    write.csv(cbind(GeneName = rownames(dat_norm), dat_norm[,tissue_diet %in% "hippo_chow"]),
              file = "hippoChow.csv",
              row.names = FALSE
    )
    write.csv(cbind(GeneName = rownames(dat_norm), dat_norm[,tissue_diet %in% "hippo_hfd"]),
              file = "hippoHFD.csv",
              row.names = FALSE
    )
    write.csv(cbind(GeneName = rownames(dat_norm), dat_norm[,tissue_diet %in% "hpt_chow"]), 
              file = "hptChow.csv",
              row.names = FALSE
    )
    write.csv(cbind(GeneName = rownames(dat_norm), dat_norm[,tissue_diet %in% "hpt_hfd"]),
              file = "hptHFD.csv", 
              row.names = FALSE
    ) 
}

In [14]:
# legend of the plot
legs <- c("chow samples", "hfd samples")

legc <- c("lightskyblue", "indianred1")
# colors for volcano plot
legv <- c("red", "grey")

# colors for circular plot of target genes
legcir_colo <- brewer.pal(10, "BrBG")

### Hippocampus Chow

In [15]:
# circadian genes using JTK
if(bool_recomp){
    hipChow <- rhythm_gene(infile = "hippoChow.csv",
                           filestyle = "csv",
                           method = "JTK",#"JTK",#tigar
                           family = "nb",
                           timepoints = as.numeric(as.character(timepoint[tissue_diet %in% "hippo_chow"])), # MetaCycle
                           #timepoints = tigar_tp[tissue_diet %in% "hippo_chow"], # tigaR
                           knot = 5, 
                           deg = 3,
                           ncpus = 4
    )
    write.csv(hipChow, file.path(dir_tab, 'circadian_genes_hippo_chow.csv')
    )
} else {
    hipChow = read.csv(file.path(dir_tab, 'circadian_genes_hippo_chow.csv'),
                       row.names = 1
    )
}
# select significant genes
hipC <- hipChow[hipChow$ADJ.P < 0.05,]

In [16]:
# enrichment of selected pathways
if(bool_recomp){
    kkHipC <-  enrichSelectPath(genes = hipC[,1],
                                selectPathway = targetGenes$KEGG,
                                cut_off = 0.05,
                                name = "hippo_chow",
                                orgdb = org.Rn.eg.db,
                                organism = 'rno'
    )
    write.csv(kkHipC, 
              file.path(dir_tab, 'kegg_enrichment_circadian_hippo_chow.csv')
    )
} else {
    kkHipC = read.csv(file.path(dir_tab, 'kegg_enrichment_circadian_hippo_chow.csv'), 
                      row.names = 1
    )
}

### Hippocampus HFD

In [17]:
# circadian genes using JTK
if(bool_recomp){
    hipHfd <- rhythm_gene(infile = "hippoHFD.csv",
                          filestyle = "csv",
                          method = "JTK",
                          family = "nb",
                          timepoints = as.numeric(as.character(timepoint[tissue_diet %in% "hippo_hfd"])), # MetaCycle
                          #timepoints = tigar_tp[tissue_diet %in% "hippo_hfd"], # tigaR
                          knot = 5, 
                          deg = 3,
                          ncpus = 4
    )
    write.csv(hipHfd, 
              file.path(dir_tab, 'circadian_genes_hippo_hfd.csv')
    )
} else {
    hipHfd = read.csv(file.path(dir_tab, 'circadian_genes_hippo_hfd.csv'),
                      row.names = 1
    )
}
# select significant genes
hipH <- hipHfd[hipHfd$ADJ.P < 0.05,]

In [18]:
# enrichment of selected pathways
if(bool_recomp){
    kkHipH <-  enrichSelectPath(genes = hipH[,1],
                                selectPathway = targetGenes$KEGG,
                                cut_off = 0.05,
                                name = "hippo_hfd",
                                orgdb = org.Rn.eg.db,
                                organism = 'rno'
    )
    write.csv(kkHipH,
              file.path(dir_tab, 'kegg_enrichment_circadian_hippo_hfd.csv')
    )
} else {
    kkHipH = read.csv(file.path(dir_tab, 'kegg_enrichment_circadian_hippo_hfd.csv'), 
                      row.names = 1
    )
}

### Hypothalmus Chow

In [19]:
# circadian genes using JTK
if(bool_recomp){
    hptChow <- rhythm_gene(infile = "hptChow.csv",
                          filestyle = "csv",
                          method = "JTK",
                          family = "nb",
                          timepoints = as.numeric(as.character(timepoint[tissue_diet %in% "hpt_chow"])),
                          #timepoints = tigar_tp[tissue_diet %in% "hpt_chow"], # tigaR
                          knot = 5, 
                          deg = 3,
                          ncpus = 4
    )
    write.csv(hptChow,
              file.path(dir_tab, 'circadian_genes_hpt_chow.csv')
    )
} else {
    hptChow = read.csv(file.path(dir_tab, 'circadian_genes_hpt_chow.csv'),
                       row.names = 1
    )
}
hptC <- hptChow[hptChow$ADJ.P < 0.05,]

In [20]:
# enrichment of selected pathways
if(bool_recomp){
    kkHptC <- enrichSelectPath(genes = hptC[,1],
                               selectPathway = targetGenes$KEGG,
                               cut_off = 0.05,
                               name = "hpt_chow",
                               orgdb = org.Rn.eg.db,
                               organism = 'rno'
    )
    write.csv(kkHptC, 
              file.path(dir_tab,'kegg_enrichment_circadian_hpt_chow.csv')
    )
} else {
    kkHptC = read.csv(file.path(dir_tab,'kegg_enrichment_circadian_hpt_chow.csv'),
                      row.names = 1
    )
}

### Hypothalmus HFD

In [21]:
# circadian genes using JTK
if(bool_recomp){
    hptHfd <- rhythm_gene(infile = "hptHFD.csv",
                          filestyle = "csv",
                          method = "JTK",
                          family = "nb",
                          timepoints = as.numeric(as.character(timepoint[tissue_diet %in% "hpt_hfd"])),
                          #timepoints = tigar_tp[tissue_diet %in% "hpt_hfd"], # tigaR
                          knot = 5, 
                          deg = 3,
                          ncpus = 4
    )
    write.csv(hptHfd, 
              file.path(dir_tab,'circadian_genes_hpt_hfd.csv')
    )
} else {
    hptHfd = read.csv(file.path(dir_tab,'circadian_genes_hpt_hfd.csv'),
                      row.names = 1
    )
}
hptH <- hptHfd[hptHfd$ADJ.P < 0.05,]

In [22]:
# enrichment of selected pathways
if(bool_recomp){
    kkHptH = enrichSelectPath(genes = hptH[,1],
                              selectPathway = targetGenes$KEGG,
                              cut_off = 0.05,
                              name = "hpt_hfd",
                              orgdb = org.Rn.eg.db,
                              organism = 'rno'
    )
    write.csv(kkHptH, 
              file.path(dir_tab,'kegg_enrichment_circadian_hpt_hfd.csv')
    )
} else{
    kkHptH = read.csv(file.path(dir_tab,'kegg_enrichment_circadian_hpt_hfd.csv'),
                      row.names = 1  
    )
}

## Visualize circadian genes 

In [23]:
# summarize data per time point using mean
sp <- paste(tissue, diet, timepoint, sep = '')

sumdat <- numeric()
for(i in unique(sp)){
    sumdat <- cbind(sumdat, rowMeans(vsd[,sp == i])) # mean of the counts per timepoint and diet
}

colnames(sumdat) <- rep(c("TP2", "TP6", "TP10", "TP14", "TP18", "TP22"), 4)

### Hippocampus 

In [24]:
# circadian genes in chow hippocampus
if(bool_plot){
    circadian_heatmap(data = list(hippochow = hipC[,1], hippohfd = hipH[, 1]),                  
                      colinf = sub("\\d.*", "", unique(sp)),
                      sumdat = sumdat,
                      title = "Hippo chow circadian genes",
                      filename = file.path(dir_fig, "heatmap_circadian_hippo_chow.pdf"),
                      col = colors,
                      distfun = dist.pear,
                      hclustfun = hclust.ave,
                      legs = legs,
                      legc = legc
    )
}

In [25]:
# circadian genes in chow hippocampus of glucose methabolism genes
if(bool_plot){
    circadian_heatmap(data = list(hippochow = intersect(hipC[,1],targetGenes$glucoseMetab),
                                  hippohfd = intersect(hipH[, 1],targetGenes$glucoseMetab)),                  
                      colinf = sub("\\d.*", "", unique(sp)),
                      sumdat = sumdat,
                      title = "Hippo chow circadian genesc - Glucose Methabolism",
                      filename = file.path(dir_fig, "heatmap_circadian_hippo_chow_glucoseMetab.pdf"),
                      col = colors,
                      distfun = dist.pear,
                      hclustfun = hclust.ave,
                      legs = legs,
                      legc = legc
    )
}

In [26]:
# circadian genes in hfd hippocampus
if(bool_plot){ 
    circadian_heatmap(data = list(hippohfd = hipH[,1], hippochow = hipC[,1]),                  
                      colinf = sub("\\d.*", "", unique(sp)),
                      sumdat = sumdat,
                      title = "Hippo hfd circadian genes",
                      filename = file.path(dir_fig, "heatmap_circadian_hippo_hfd.pdf"),
                      col = colors,
                      distfun = dist.pear,
                      hclustfun = hclust.ave,
                      legs = legs,
                      legc = legc
    )
}

In [27]:
# circadian genes in hfd hippocampus
if(bool_plot){ 
    circadian_heatmap(data = list(hippohfd = intersect(hipH[,1],targetGenes$glucoseMetab), 
                                  hippochow = intersect(hipC[,1],targetGenes$glucoseMetab)),                  
                      colinf = sub("\\d.*", "", unique(sp)),
                      sumdat = sumdat,
                      title = "Hippo hfd circadian genes - Glucose Methabolism",
                      filename = file.path(dir_fig, "heatmap_circadian_hippo_hfd_glucoseMetab.pdf"),
                      col = colors,
                      distfun = dist.pear,
                      hclustfun = hclust.ave,
                      legs = legs,
                      legc = legc
    )
}

### Hypothalamus

In [28]:
# circadian genes in chow hypothalamus
if(bool_plot){
    circadian_heatmap(data = list(hptchow = hptC[,1], hpthfd = hptH[,1]),                  
                      colinf = sub("\\d.*", "", unique(sp)),
                      sumdat = sumdat,
                      title = "Hpt chow circadian genes",
                      filename = file.path(dir_fig, "heatmap_circadian_hpt_chow.pdf"),
                      col = colors,
                      distfun = dist.pear,
                      hclustfun = hclust.ave,
                      legs = legs,
                      legc = legc
    )
}

In [29]:
# circadian genes in chow hypothalamus
if(bool_plot){
    circadian_heatmap(data = list(hptchow = intersect(hptC[,1], targetGenes$glucoseMetab), 
                                  hpthfd = intersect(hptH[,1], targetGenes$glucoseMetab)),                  
                      colinf = sub("\\d.*", "", unique(sp)),
                      sumdat = sumdat,
                      title = "Hpt chow circadian genes - Glucose Methabolism",
                      filename = file.path(dir_fig, "heatmap_circadian_hpt_chow_glucoseMetab.pdf"),
                      col = colors,
                      distfun = dist.pear,
                      hclustfun = hclust.ave,
                      legs = legs,
                      legc = legc
    )
}

In [30]:
# circadian genes in hfd hypothalamus
if(bool_plot){ 
    circadian_heatmap(data = list(hpthfd = hptH[,1], hptchow = hptC[,1]),                  
                      colinf = sub("\\d.*", "", unique(sp)),
                      sumdat = sumdat,
                      title = "Hpt hfd circadian genes",
                      filename = file.path(dir_fig,"heatmap_circadian_hpt_hfd.pdf"),
                      col = colors,
                      distfun = dist.pear,
                      hclustfun = hclust.ave,
                      legs = legs,
                      legc = legc
    )
}

In [31]:
# circadian genes in hfd hypothalamus
if(bool_plot){ 
    circadian_heatmap(data = list(hpthfd = intersect(hptH[,1], targetGenes$glucoseMetab),
                                  hptchow = intersect(hptC[,1], targetGenes$glucoseMetab)),                  
                      colinf = sub("\\d.*", "", unique(sp)),
                      sumdat = sumdat,
                      title = "Hpt hfd circadian genes - Glucose Methabolism",
                      filename = file.path(dir_fig,"heatmap_circadian_hpt_hfd_glucoseMetab.pdf"),
                      col = colors,
                      distfun = dist.pear,
                      hclustfun = hclust.ave,
                      legs = legs,
                      legc = legc
    )
}

In [32]:
# Enrichment analysis of selected pathways
if(bool_plot){
    enrich <- ggplot(rbind(kkHipC, kkHipH, kkHptC, kkHptH), 
                     aes(Comparison, Description, col = p.adjust, size = Count)) + 
                     geom_point()+ 
                     scale_colour_gradient(low = "red", high = "blue") +
                     xlab("caricadian genes") + 
                     ylab("enriched pathways")
    
    ggsave(file.path(dir_fig, "kegg_enrichment_selected_pathawys_circadian_genes.pdf"))
    print(enrich)
}

In [33]:
# venn diagrams of hypothalamys and hippocampus
if(bool_plot){
    hipA <- rbind(data.frame(amplitude = hipC$AMP, diet = 'chow'), data.frame(amplitude = hipH$AMP, diet = 'hfd'))
    hptA <- rbind(data.frame(amplitude = hptC$AMP, diet = 'chow'), data.frame(amplitude = hptH$AMP, diet = 'hfd'))
    figure <- multi_panel_figure(columns = 2,
                                 rows = 2
              )
    figure %<>% fill_panel(venn.diagram(list(' '= hipC[,1],' '= hipH[,1]), 
                                        fill = legc,
                                        filename = NULL,
                                        col = "transparent",
                                        main = "Hippocampus",
                                        rotation.degree = 270
                  )
        ) %<>% fill_panel(venn.diagram(list(' '= hptC[,1],' '= hptH[,1]), 
                                       fill = legc,
                                       filename = NULL,
                                       col = "transparent",
                                       main = "Hypothalamus",
                                       rotation.degree = 270
                  )
        ) %<>% fill_panel(ggplot(hipA, aes(amplitude, fill = diet)) + 
                                 geom_histogram(alpha = 0.7, aes(y = ..density..), position = 'identity', bins = 20) +
                                 scale_fill_manual(values = legc) +
                                 ggtitle("Hippocampus")
        ) %<>% fill_panel(ggplot(hptA, aes(amplitude, fill = diet)) + 
                                 geom_histogram(alpha = 0.7, aes(y = ..density..), position = 'identity', bins = 20) + 
                                 scale_fill_manual(values = legc) + 
                                 ggtitle("Hypothalamus")
        )
    save_multi_panel_figure(figure, 
                            filename = file.path(dir_fig,"venn_diagram-amplitude_circadian_genes.pdf")
    )
    figure
}

In [34]:
# intersection of overlaping genes
if(bool_plot){  
    pdf(file = file.path(dir_fig, "upset_plot_circadian_genes.pdf"))
    ups <- plotUpset(analyses = list(hipChow = hipChow,
                                     hipHfd = hipHfd,
                                     hptChow = hptChow, 
                                     hptHfd = hptHfd
                     ),
                     cut_off = 0.05,
                     col_interest = 3,
                     text.scale = 2,
                     matrix.color = "black",
                     main.bar.color = "black",
                     sets.bar.color = "black"
    )
    dev.off()
    ups
}

## Differential expression - tissue and diet effect

### Tissue effect

In [35]:
# differntial gene expression
if(bool_recomp){
    dds_t <- DESeq(dds,
                   test = "LRT",
                   reduced = ~ timepoint + diet
             )
    tissue_effect <- as.data.frame(results(dds_t))
    write.csv(tissue_effect, 
              file.path(dir_tab, 'differential_expression_tissue_effect.csv')
    )
} else {
    tissue_effect = read.csv(file.path(dir_tab, 'differential_expression_tissue_effect.csv'),
                             row.names = 1
    )
}
# select significant genes
hiphpt <- tissue_effect[tissue_effect[,6] < 0.01,]

In [36]:
# enrichment of selected pathways
if(bool_recomp){
    kkHipHpt <- enrichSelectPath(genes = rownames(hiphpt),
                                 selectPathway = targetGenes$KEGG,
                                 cut_off = 0.05,
                                 name = "tissue_effect",
                                 orgdb = org.Rn.eg.db,
                                 organism = 'rno'
    )
    write.csv(kkHipHpt, 
              file.path(dir_tab, 'kegg_enrichment_tissue_effect.csv')
    )
} else {
    kkHipHpt = read.csv(file.path(dir_tab, 'kegg_enrichment_tissue_effect.csv'),
                        row.names = 1 
    )
}

In [37]:
# volcano plot differential expressed genes
if(bool_plot){ 
    res = tissue_effect[order(tissue_effect$padj),]
    results = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                          significant = ifelse(res$padj < 0.05, "FDR<0.05", "Not Sig")
                                         ), row.names = rownames(res)
                           )
    results = results[(!is.na(results$padj)),] 

    vdhip <- ggplot(results, aes(log2FoldChange, -log10(padj))) +
                    geom_point(aes(col = significant)) +
                    scale_color_manual(values = legv) +
                    geom_text_repel(data = results[1:20,], aes(label = rownames(results[1:20,])))
    ggsave(file.path(dir_fig, "volcano_differential_expression_tissue_effect.pdf"))
    print(vdhip)
}

In [38]:
# separate condition per tissue and make metadata
t_hip <- tissue %in% "hippo"
t_hpt <- tissue %in% "hpt"
timepoint1 <- timepoint[t_hip]
timepoint2 <- timepoint[t_hpt]
diet1 <- diet[t_hip]
diet2 <- diet[t_hpt]
id1 <- colnames(dat_filt[,t_hip])
id2 <- colnames(dat_filt[,t_hpt])
metaData1 <- data.frame(id1, timepoint1, diet1)
metaData2 <- data.frame(id2, timepoint2, diet2)

### Hippocampus - Diet Effect 

In [39]:
# differntial gene expression
if(bool_recomp){
    ddsHip <- DESeqDataSetFromMatrix(countData = dat_filt[,t_hip],
                                     colData = metaData1,
                                     design =~ timepoint1 + diet1
              )
    ddsHip <- DESeq(ddsHip,
                    test = "LRT",
                    reduced = ~ timepoint1 
              )
    hipDiet <- as.data.frame(results(ddsHip))
    write.csv(hipDiet, 
              file.path(dir_tab, 'differential_expression_hippo_diet_effect.csv')
    )
} else {
    hipDiet = read.csv(file.path(dir_tab, 'differential_expression_hippo_diet_effect.csv'),
                       row.names = 1
    )
}
# select significant genes
hipD <- hipDiet[hipDiet[,6] < 0.01,]

In [40]:
# enrichment of selected pathways
if(bool_recomp){
    kkHipD <- enrichSelectPath(genes = rownames(hipD),
                               selectPathway = targetGenes$KEGG,
                               cut_off = 0.05,
                               name = "hippo_diet_effect",
                               orgdb = org.Rn.eg.db,
                               organism = 'rno'
    )
    write.csv(kkHipD,
              file.path(dir_tab, 'kegg_enrichment_hippo_diet_effect.csv')
    )
} else {
    kkHipD = read.csv(file.path(dir_tab, 'kegg_enrichment_hippo_diet_effect.csv'),
                      row.names = 1,
                      sep = ','
    )
}

In [41]:
# volcano plot differential expressed genes
if(bool_plot){ 
    res = hipDiet[order(hipDiet$padj),]
    results = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                          significant = ifelse(res$padj < 0.05, "FDR<0.05", "Not Sig")
                                         ), row.names = rownames(res)
                           )
    results = results[(!is.na(results$padj)),] 

    vdhip <- ggplot(results, aes(log2FoldChange, -log10(padj))) +
                    geom_point(aes(col = significant)) +
                    scale_color_manual(values = legv) +
                    geom_text_repel(data = results[1:20,], aes(label = rownames(results[1:20,])))
    ggsave(file.path(dir_fig, "volcano_differential_expression_hippo_diet_effect.pdf"))
    print(vdhip)
}

### Hypothalamus - Diet Effect

In [42]:
# differntial gene expression
if(bool_recomp){
    ddsHpt <- DESeqDataSetFromMatrix(countData = dat_filt[,t_hpt],
                                     colData = metaData2,
                                     design =~ timepoint2 + diet2
              )
    ddsHpt <- DESeq(ddsHpt,
                    test = "LRT",
                    reduced = ~ timepoint2
              )
    hptDiet <- as.data.frame(results(ddsHpt))
    write.csv(hptDiet,
              file.path(dir_tab, 'differential_expression_hpt_diet_effect.csv')
    )
} else {
    hptDiet = read.csv(file.path(dir_tab, 'differential_expression_hpt_diet_effect.csv'),
                       row.names = 1
    )
}
# select significant genes
hptD <- hptDiet[hptDiet[,6] < 0.01,]

In [43]:
# enrichment of selected pathways
if(bool_recomp){
    kkHptD <- enrichSelectPath(genes = rownames(hptD),
                               selectPathway = targetGenes$KEGG,
                               cut_off = 0.05,
                               name = "hpt_diet_effect",
                               orgdb = org.Rn.eg.db,
                               organism = 'rno'
    )
    write.csv(kkHptD,
              file.path(dir_tab, 'kegg_enrichment_hpt_diet_effect.csv')
    )
} else {
    kkHptD = read.csv(file.path(dir_tab, 'kegg_enrichment_hpt_diet_effect.csv'),
                      row.names = 1,
                      sep = ','
    )
}

In [44]:
# volcano plot differential expressed genes
if(bool_plot){ 
    res = hptDiet[order(hptDiet$padj),]
    results = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                          significant = ifelse(res$padj < 0.05, "FDR<0.05", "Not Sig")
                                         ), row.names = rownames(res)
                           )
    results = results[(!is.na(results$padj)),] 

    vdhpt <- ggplot(results, aes(log2FoldChange, -log10(padj))) +
                    geom_point(aes(col = significant)) +
                    scale_color_manual(values = legv) +
                    geom_text_repel(data = results[1:20,], aes(label = rownames(results[1:20,])))
    ggsave(file.path(dir_fig, "volcano_differential_expression_hpt_diet_effect.pdf"))
    print(vdhpt)
}

In [45]:
# Enrichment analysis of selected pathways
if(bool_plot){
    enrich <- ggplot(rbind(kkHipHpt, kkHipD, kkHptD), 
                     aes(Comparison,Description, col = p.adjust, size = Count)) + 
                     geom_point()+ 
                     scale_colour_gradient(low = "red", high = "blue") +
                     xlab("caricadian genes") + 
                     ylab("enriched pathways")
    
    ggsave(file.path(dir_fig, "kegg_enrichment_tissue_diet_effect_selected_pathawys.pdf"))
    print(enrich)
}

In [46]:
# intersection of overlaping genes
if(bool_plot){  
    pdf(file = file.path(dir_fig, "upset_differential_genes_tissue_diet_effect.pdf"))
   
    ups <- plotUpset(analyses = list(hipDiet = hipDiet, 
                                     hptDiet = hptDiet, 
                                     tissue_effect = tissue_effect
                     ),
                     cut_off = 0.05,
                     col_interest = 6,
                     text.scale = 2,
                     matrix.color = "black",
                     main.bar.color = "black",
                     sets.bar.color = "black"
    )
    dev.off()
    ups
}


In [47]:
# plot individualy genes of interest
if(bool_plot){
    pdf(file = file.path(dir_fig, "time_plot_hpt_hippo_interesting_genes.pdf"))
    for(i in targetGenes$hpt_hip_genes){
        circadian_gene_plot(gene_name = i,
                            data1 = vsd[,t_hip],
                            tp1 = timepoint1,
                            group1 = diet1,
                            data2 = vsd[,t_hpt],
                            tp2 = timepoint2,
                            group2 = diet2
    )
    }
    dev.off()
}

In [48]:
# plot individual gene 
if(bool_plot){
    circadian_gene_plot(gene_name = "Clock",
                        data1 = vsd[,t_hip],
                        tp1 = timepoint1,
                        group1 = diet1,
                        data2 = vsd[,t_hpt],
                        tp2 = timepoint2,
                        group2 = diet2
    )
}

## Differential experssion per time point

In [49]:
# differential gene expression analysis per time point comparing chow vs. hfd
dds <- DESeqDataSetFromMatrix(countData = dat_filt,
                              colData = metaData,
                              design =~ condition
)
dds <- DESeq(dds)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



### Hippocampus TP2

In [50]:
# differntial gene expression
if(bool_recomp){
    hippoTP2 <- shrink_dge(dds = dds,
                           method = "deseq2",
                           contrast = c("condition", "hippo2chow", "hippo2hfd"),
    )
    hippoTP2[is.na(hippoTP2$padj),]  <- 1
    write.csv(hippoTP2, 
              file.path(dir_tab, 'differential_expression_hippo_TP2.csv')
    )
} else {
    hippoTP2 = read.csv(file.path(dir_tab, 'differential_expression_hippo_TP2.csv'),
                        row.names = 1
    )
}
# select significant genes
hip2 <- hippoTP2[hippoTP2[,6] < 0.05,]

In [51]:
# enrichment of selected pathways
if(bool_recomp){
    kkHip2 <- enrichSelectPath(genes = rownames(hip2),
                               selectPathway = targetGenes$KEGG,
                               cut_off = 0.05,
                               name = "hippo_TP2",
                               orgdb = org.Rn.eg.db,
                               organism = 'rno'
    )
    write.csv(kkHip2, 
              file.path(dir_tab, 'kegg_enrichment_hippo_TP2.csv')
    )
} else {
    kkHip2 = read.csv(file.path(dir_tab, 'kegg_enrichment_hippo_TP2.csv'),
                      row.names = 1,
                      sep = ','
    )
}

In [52]:
# volcano plot differential expressed genes
if(bool_plot){ 
    res = hippoTP2[order(hippoTP2$padj),]
    results = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                          significant = ifelse(res$padj < 0.05, "FDR<0.05", "Not Sig")
                                    ), row.names=rownames(res)
    )
    vdhpt <- ggplot(results, aes(log2FoldChange, -log10(padj))) +
                    geom_point(aes(col = significant)) +
                    scale_color_manual(values = legv) +
                    geom_text_repel(data = results[1:20,], aes(label = rownames(results[1:20,])))
    ggsave(file.path(dir_fig, "volcano_hippo_TP2.pdf"))
    print(vdhpt)
}

### Hippocampus TP6

In [53]:
# differntial gene expression
if(bool_recomp){
    hippoTP6 <- shrink_dge(dds = dds,
                           method = "deseq2",
                           contrast = c("condition", "hippo6chow", "hippo6hfd")
    )
    hippoTP6[is.na(hippoTP6$padj),]  <- 1
    write.csv(hippoTP6, 
              file.path(dir_tab,'differential_expression_hippo_TP6.csv')
    )
} else {
    hippoTP6 = read.csv(file.path(dir_tab,'differential_expression_hippo_TP6.csv'),
                        row.names = 1
    )
}
# select significant genes
hip6 <- hippoTP6[hippoTP6[,6] < 0.05,]

In [54]:
# enrichment of selected pathways
if(bool_recomp){
    kkHip6 <- enrichSelectPath(genes = rownames(hip6),
                               selectPathway = targetGenes$KEGG,
                               cut_off = 0.05,
                               name = "hippo_TP6",
                               orgdb = org.Rn.eg.db,
                               organism = 'rno'
    )
    write.csv(kkHip6, 
              file.path(dir_tab, 'kegg_enrichment_hippo_TP6.csv')
    )
} else {
    kkHip6 = read.table(file.path(dir_tab,'kegg_enrichment_hippo_TP6.csv'),
                        row.names = 1,
                        sep = ','
    )
}

In [55]:
# volcano plot differential expressed genes
if(bool_plot){ 
    res = hippoTP6[order(hippoTP6$padj),]
    results = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                          significant = ifelse(res$padj < 0.05, "FDR<0.05", "Not Sig")
                                   ), row.names = rownames(res)
    )
    vdhpt <- ggplot(results, aes(log2FoldChange, -log10(padj))) +
                    geom_point(aes(col = significant)) +
                    scale_color_manual(values = legv) +
                    geom_text_repel(data = results[1:10,], aes(label = rownames(results[1:10,])))
    ggsave(file.path(dir_fig, "volcano_hippo_TP6.pdf"))
    print(vdhpt)
}

### Hippocampus TP10

In [56]:
# differntial gene expression
if(bool_recomp){
    hippoTP10 <- shrink_dge(dds = dds,
                            method = "deseq2",
                            contrast = c("condition", "hippo10chow", "hippo10hfd")  
    )
    hippoTP10[is.na(hippoTP10$padj),]  <- 1
    write.csv(hippoTP10, 
              file.path(dir_tab,'differential_expression_hippo_TP10.csv')
    )
} else {
    hippoTP10 = read.csv(file.path(dir_tab,'differential_expression_hippo_TP10.csv'),
                         row.names = 1
    )
}
# select significant genes
hip10 <- hippoTP10[hippoTP10[,6] < 0.05,]

In [57]:
# enrichment of selected pathways
if(bool_recomp){
    kkHip10 <- enrichSelectPath(genes = rownames(hip10),
                               selectPathway = targetGenes$KEGG,
                               cut_off = 0.05,
                               name = "hippo_TP10",
                               orgdb = org.Rn.eg.db,
                               organism = 'rno'
    )
    write.csv(kkHip10, 
              file.path(dir_tab, 'kegg_enrichment_hippo_TP10.csv')
    )
} else {
    kkHip10 = read.csv(file.path(dir_tab, 'kegg_enrichment_hippo_TP10.csv'), 
                       row.names = 1,
                       sep = ',' 
    )
}

In [58]:
# volcano plot differential expressed genes
if(bool_plot){ 
    res = hippoTP10[order(hippoTP10$padj),]
    results = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                          significant = ifelse(res$padj < 0.05, "FDR<0.05", "Not Sig")
                                   ), row.names=rownames(res)
    )
    vdhpt <- ggplot(results, aes(log2FoldChange, -log10(padj))) +
                    geom_point(aes(col = significant)) +
                    scale_color_manual(values = legv) +
                    geom_text_repel(data = results[1:20,], aes(label = rownames(results[1:20,])))
    ggsave(file.path(dir_fig, "volcano_hippo_TP10.pdf"))
    print(vdhpt)
}

### Hippocampus TP14

In [59]:
# differntial gene expression
if(bool_recomp){
    hippoTP14 <- shrink_dge(dds = dds,
                            method = "deseq2",
                            contrast = c("condition", "hippo14chow", "hippo14hfd")   
    )
    hippoTP14[is.na(hippoTP14$padj),]  <- 1
    write.csv(hippoTP14,
              file.path(dir_tab, 'differential_expression_hippo_TP14.csv')
    )
} else {
    hippoTP14 = read.csv(file.path(dir_tab, 'differential_expression_hippo_TP14.csv'),
                         row.names = 1
    )
}
# select significant genes
hip14 <- hippoTP14[hippoTP14[,6] < 0.05,]

In [60]:
# enrichment of selected pathways
if(bool_recomp){
    kkHip14 <- enrichSelectPath(genes = rownames(hip14),
                                selectPathway = targetGenes$KEGG,
                                cut_off = 0.05,
                                name = "hippo_TP14",
                                orgdb = org.Rn.eg.db,
                                organism = 'rno'
    )
    write.csv(kkHip14, 
              file.path(dir_tab, 'kegg_enrichment_hippo_TP14.csv')
    )
} else {
    kkHip14 = read.csv(file.path(dir_tab, 'kegg_enrichment_hippo_TP14.csv'),
                       row.names = 1,
                       sep = ',' 
    )
}

In [61]:
# volcano plot differential expressed genes
if(bool_plot){ 
    res = hippoTP14[order(hippoTP14$padj),]
    results = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                          significant = ifelse(res$padj < 0.05, "FDR<0.05", "Not Sig")
                                   ), row.names=rownames(res)
    )
    vdhpt <- ggplot(results, aes(log2FoldChange, -log10(padj))) +
                    geom_point(aes(col = significant)) +
                    scale_color_manual(values = legv) +
                    geom_text_repel(data = results[1:20,], aes(label = rownames(results[1:20,])))
    ggsave(file.path(dir_fig, "volcano_hippo_TP14.pdf"))
    print(vdhpt)
}

### Hippocampus TP18

In [62]:
# differntial gene expression
if(bool_recomp){
    hippoTP18 <- shrink_dge(dds = dds,
                            method = "deseq2",
                            contrast = c("condition", "hippo18chow", "hippo18hfd")
    )
    hippoTP18[is.na(hippoTP18$padj),]  <- 1
    write.csv(hippoTP18, 
              file.path(dir_tab,'differential_expression_hippo_TP18.csv')
    )
} else {
    hippoTP18 = read.csv(file.path(dir_tab,'differential_expression_hippo_TP18.csv'),
                         row.names = 1
    )
}
# select significant genes
hip18 <- hippoTP18[hippoTP18[,6] < 0.05,]

In [63]:
# enrichment of selected pathways
if(bool_recomp){
    kkHip18 <- enrichSelectPath(genes = rownames(hip18),
                               selectPathway = targetGenes$KEGG,
                               cut_off = 0.05,
                               name = "hippo_TP18",
                               orgdb = org.Rn.eg.db,
                               organism = 'rno'
    )
    write.csv(kkHip18,
              file.path(dir_tab,'kegg_enrichment_hippo_TP18.csv')
    )
} else {
    kkHip18 = read.csv(file.path(dir_tab,'kegg_enrichment_hippo_TP18.csv'),
                       row.names = 1,
                       sep = ',' 
    )
}

In [64]:
# volcano plot differential expressed genes
if(bool_plot){ 
    res = hippoTP18[order(hippoTP18$padj),]
    results = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                          significant = ifelse(res$padj<0.05, "FDR<0.05", "Not Sig")
                                   ), row.names=rownames(res)
    )
    vdhpt <- ggplot(results, aes(log2FoldChange, -log10(padj))) +
                    geom_point(aes(col = significant)) +
                    scale_color_manual(values = legv) +
                    geom_text_repel(data = results[1:20,], aes(label = rownames(results[1:20,])))
    ggsave(file.path(dir_fig, "volcano_hippo_TP18.pdf"))
    print(vdhpt)
}

### Hippocampus TP22

In [65]:
# differntial gene expression
if(bool_recomp){
    hippoTP22 <- shrink_dge(dds = dds,
                            method = "deseq2",
                            contrast = c("condition", "hippo22chow", "hippo22hfd")       
                )
    hippoTP22[is.na(hippoTP22$padj),]  <- 1
    write.csv(hippoTP22,
              file.path(dir_tab, 'differential_expression_hippo_TP22.csv'))
} else {
    hippoTP22 = read.csv(file.path(dir_tab, 'differential_expression_hippo_TP22.csv'),
                         row.names = 1
    )
}
# select significant genes
hip22 <- hippoTP22[hippoTP22[,6] < 0.05,]

In [66]:
# enrichment of selected pathways
if(bool_recomp){
    kkHip22 <- enrichSelectPath(genes = rownames(hip22),
                               selectPathway = targetGenes$KEGG,
                               cut_off = 0.05,
                               name = "hippo_TP22",
                               orgdb = org.Rn.eg.db,
                               organism = 'rno'
    )
    write.csv(kkHip22,
              file.path(dir_tab, 'kegg_enrichment_hippo_TP22.csv')
    )
} else {
    kkHip22 = read.csv(file.path(dir_tab, 'kegg_enrichment_hippo_TP22.csv'),
                       row.names = 1,
                       sep = ','
    )
}

In [67]:
# volcano plot differential expressed genes
if(bool_plot){ 
    res = hippoTP22[order(hippoTP22$padj),]
    results = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                          significant = ifelse(res$padj < 0.05, "FDR<0.05", "Not Sig")
                                   ), row.names = rownames(res)
    )
    vdhpt <- ggplot(results, aes(log2FoldChange, -log10(padj))) +
                    geom_point(aes(col = significant)) +
                    scale_color_manual(values = legv) +
                    geom_text_repel(data = results[1:20,], aes(label = rownames(results[1:20,])))
    ggsave(file.path(dir_fig, "volcano_hippo_TP22.pdf"))
    print(vdhpt)
}

In [68]:
# intersection of overlaping genes
if(bool_plot){  
    pdf(file = file.path(dir_fig, "upset_differential_genes_hippo_per_TP.pdf"))

    ups <- plotUpset(analyses = list(hippoTP2 = hippoTP2, 
                                     hippoTP6 = hippoTP6, 
                                     hippoTP10 = hippoTP10, 
                                     hippoTP14 = hippoTP14,
                                     hippoTP18 = hippoTP18,
                                     hippoTP22 = hippoTP22),
                     cut_off = 0.05,
                     col_interest = 6,
                     text.scale = 2,
                     matrix.color = "black",
                     main.bar.color = "black",
                     sets.bar.color = "black"
    )
    dev.off()
    ups
}


In [69]:
# bar plot of the number of significant genes per time point in hippocampus
if(bool_plot){
    df <- data.frame(Time = c("TP1_2h", "TP2_6h", "TP3_10h", "TP4_14h", "TP5_18h", "TP6_22h"),
                     Nr_significant_genes = c(nrow(hip2), nrow(hip6), nrow(hip10), nrow(hip14), nrow(hip18), nrow(hip22))
    )
    hiptp <- ggplot(df, aes(x = Time, y = Nr_significant_genes, fill = Time)) +
                    geom_bar(stat = "identity") + 
                    theme_minimal() +
                    theme(legend.position = "none") +
                    scale_fill_manual(values = c(rep('#E6E6E6', 3), rep("#4D4D4D", 3))) +
    
    ggsave(file.path(dir_fig, "barplot_per_timepoint_hippo.pdf"))
    print(hiptp)
}

In [70]:
# Enrichment analysis of selected pathways
if(bool_plot){
    enrich <- ggplot(rbind(kkHip2, kkHip14, kkHip18), 
                     aes(Comparison, Description, col = p.adjust, size = Count)) + 
                     geom_point() + 
                     scale_colour_gradient(low = "red", high = "blue") +
                     xlab("caricadian genes") + 
                     ylab("enriched pathways")
    
    ggsave(file.path(dir_fig, "kegg_enrichment_selected_pathawys.pdf"))
    print(enrich)
}

In [71]:
# circular visualization of volcano plots - hippocampus
if(bool_plot){ 
    v_hip <- rbind(cbind(hippoTP2, comp = "Hippo_TP2"),
                   cbind(hippoTP6, comp = "Hippo_TP6"),
                   cbind(hippoTP10, comp = "Hippo_TP10"),
                   cbind(hippoTP14, comp = "Hippo_TP14"),
                   cbind(hippoTP18, comp = "Hippo_TP18"),
                   cbind(hippoTP22, comp = "Hippo_TP22")
    )
    v_hip$col <- ifelse(v_hip$padj > 0.05, "grey", "red")
    v_hip[,'padj'] <- -log10(v_hip[,'padj'])
    
    circular_plot(x = v_hip$log2FoldChange,
                  y = v_hip$padj, 
                  factors = v_hip$comp,
                  color = v_hip$col,
                  leg_name = c('FDR<0.05', 'Not Sig'),
                  leg_col = legv
    )
}

### Hypothalamus TP2

In [72]:
# differntial gene expression
if(bool_recomp){
    hptTP2 <- results(dds,
                      contrast = c("condition", "hpt2chow", "hpt2hfd")
    )
    hptTP2[is.na(hptTP2$padj),]  <- 1
    write.csv(hptTP2, 
              file.path(dir_tab, 'differential_expression_hpt_TP2.csv')
    )
} else {
    hptTP2 = read.csv(file.path(dir_tab, 'differential_expression_hpt_TP2.csv'),
                      row.names = 1
    )
}
# select significant genes
hpt2 <- hptTP2[hptTP2[,6] < 0.05,]

In [73]:
# enrichment of selected pathways
if(bool_recomp){
    kkHpt2 <- enrichSelectPath(genes = rownames(hpt2),
                               selectPathway = targetGenes$KEGG,
                               cut_off = 0.05,
                               name = "hpt_TP2",
                               orgdb = org.Rn.eg.db,
                               organism = 'rno'
    )
    write.csv(kkHpt2, 
              file.path(dir_tab, 'kegg_enrichment_hpt_TP2.csv')
    )
} else {
    kkHpt2 = read.csv(file.path(dir_tab, 'kegg_enrichment_hpt_TP2.csv'),
                      row.names = 1,
                      sep = ','
    )
}

In [74]:
# volcano plot differential expressed genes
if(bool_plot){ 
    res = hptTP2[order(hptTP2$padj),]
    results = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                          significant = ifelse(res$padj < 0.05, "FDR<0.05", "Not Sig")
                                   ), row.names = rownames(res)
    ) 
    vdhpt <- ggplot(results, aes(log2FoldChange, -log10(padj))) +
                    geom_point(aes(col = significant)) +
                    scale_color_manual(values = legv) +
                    geom_text_repel(data = results[1:20,], aes(label = rownames(results[1:20,])))
    ggsave(file.path(dir_fig, "volcano_hpt_TP2.pdf"))
    print(vdhpt)
}

### Hypothalamus TP6

In [75]:
# differntial gene expression
if(bool_recomp){
    hptTP6 <- shrink_dge(dds = dds,
                        method = "deseq2",
                        contrast = c("condition", "hpt6chow", "hpt6hfd")
    )
    hptTP6[is.na(hptTP6$padj),]  <- 1
    write.csv(hptTP6, 
              file.path(dir_tab, 'differential_expression_hpt_TP6.csv')
    )
} else {
    hptTP6 = read.csv(file.path(dir_tab, 'differential_expression_hpt_TP6.csv'), 
                      row.names = 1
    )
}
# select significant genes
hpt6 <- hptTP6[hptTP6[,6] < 0.05,]

In [76]:
# enrichment of selected pathways
if(bool_recomp){
    kkHpt6 <- enrichSelectPath(genes = rownames(hpt6),
                               selectPathway = targetGenes$KEGG,
                               cut_off = 0.05,
                               name = "hpt_TP6",
                               orgdb = org.Rn.eg.db,
                               organism = 'rno'
    )
    write.csv(kkHpt6,
              file.path(dir_tab, 'kegg_enrichment_hpt_TP6.csv')
    )
} else {
    kkHpt6 = read.csv(file.path(dir_tab, 'kegg_enrichment_hpt_TP6.csv'),
                      row.names = 1,
                      sep = ','
    )
}

In [77]:
# volcano plot differential expressed genes
if(bool_plot){ 
    res = hptTP6[order(hptTP6$padj),]
    results = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                          significant = ifelse(res$padj < 0.05, "FDR<0.05", "Not Sig")
                                   ), row.names = rownames(res)
    )
    vdhpt <- ggplot(results, aes(log2FoldChange, -log10(padj))) +
                    geom_point(aes(col = significant)) +
                    scale_color_manual(values = legv) +
                    geom_text_repel(data = results[1:20,], aes(label = rownames(results[1:20,])))
    ggsave(file.path(dir_fig, "volcano_hpt_TP6.pdf"))
    print(vdhpt)
}

### Hypothalamus TP10

In [78]:
# differntial gene expression
if(bool_recomp){
    hptTP10 <- shrink_dge(dds = dds,
                          method = "deseq2",
                          contrast = c("condition", "hpt10chow", "hpt10hfd")
    )
    hptTP10[is.na(hptTP10$padj),]  <- 1
    write.csv(hptTP10, file.path(dir_tab, 'differential_expression_hpt_TP10.csv'))
} else {
    hptTP10 = read.csv(file.path(dir_tab, 'differential_expression_hpt_TP10.csv'), 
                       row.names = 1
    )
}
# select significant genes
hpt10 <- hptTP10[hptTP10[,6] < 0.05,]

In [79]:
# enrichment of selected pathways
if(bool_recomp){
    kkHpt10 <- enrichSelectPath(genes = rownames(hpt10),
                               selectPathway = targetGenes$KEGG,
                               cut_off = 0.05,
                               name = "hpt_TP10",
                               orgdb = org.Rn.eg.db,
                               organism = 'rno'
    )
    write.csv(kkHpt10, file.path(dir_tab, 'kegg_enrichment_hpt_TP10.csv'))
} else {
    kkHpt10 = read.csv(file.path(dir_tab, 'kegg_enrichment_hpt_TP10.csv'),
                       row.names = 1,
                       sep = ','
    )
}

In [80]:
# volcano plot differential expressed genes
if(bool_plot){ 
    res = hptTP10[order(hptTP10$padj),]
    results = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                          significant = ifelse(res$padj < 0.05, "FDR<0.05", "Not Sig")
                                   ), row.names = rownames(res)
    )
    vdhpt <- ggplot(results, aes(log2FoldChange, -log10(padj))) +
                    geom_point(aes(col = significant)) +
                    scale_color_manual(values = legv) +
                    geom_text_repel(data = results[1:20,], aes(label = rownames(results[1:20,])))
    ggsave(file.path(dir_fig,"volcano_hpt_TP10.pdf"))
    print(vdhpt)
}

### Hypothalamus TP14

In [81]:
# differntial gene expression
if(bool_recomp){
    hptTP14 <- shrink_dge(dds = dds,
                          method = "deseq2",
                          contrast = c("condition", "hpt14chow", "hpt14hfd")
    )
    hptTP14[is.na(hptTP14$padj),]  <- 1
    write.csv(hptTP14, 
              file.path(dir_tab,'differential_expression_hpt_TP14.csv')
    )
} else {
    hptTP14 = read.csv(file.path(dir_tab,'differential_expression_hpt_TP14.csv'),
                       row.names = 1
    )
}
# select significant genes
hpt14 <- hptTP14[hptTP14[,6]<0.05,]

In [82]:
# enrichment of selected pathways
if(bool_recomp){
    kkHpt14 <- enrichSelectPath(genes = rownames(hpt14),
                               selectPathway = targetGenes$KEGG,
                               cut_off = 0.05,
                               name = "hpt_TP14",
                               orgdb = org.Rn.eg.db,
                               organism = 'rno'
    )
    write.csv(kkHpt14, 
              file.path(dir_tab, 'kegg_enrichment_hpt_TP14.csv')
    )
} else {
    kkHpt14 = read.csv(file.path(dir_tab, 'kegg_enrichment_hpt_TP14.csv'),
                       row.names = 1,
                       sep = ','
    )
}

In [83]:
# volcano plot differential expressed genes
if(bool_plot){ 
    res = hptTP14[order(hptTP14$padj),]
    results = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                          significant = ifelse(res$padj < 0.05, "FDR<0.05", "Not Sig")
                                   ), row.names = rownames(res)
    )
    vdhpt <- ggplot(results, aes(log2FoldChange, -log10(padj))) +
                    geom_point(aes(col = significant)) +
                    scale_color_manual(values = legv) +
                    geom_text_repel(data = results[1:20,], aes(label = rownames(results[1:20,])))
    ggsave(file.path(dir_fig, "volcano_hpt_TP14.pdf"))
    print(vdhpt)
}

### Hypothalamus TP18

In [84]:
# differntial gene expression
if(bool_recomp){
    hptTP18 <- shrink_dge(dds = dds,
                          method = "deseq2",
                          contrast = c("condition", "hpt18chow", "hpt18hfd")
    )
    hptTP18[is.na(hptTP18$padj),]  <- 1
    write.csv(hptTP14, 
              file.path(dir_tab, 'differential_expression_hpt_TP18.csv')
    )
} else {
    hptTP18 = read.csv(file.path(dir_tab, 'differential_expression_hpt_TP18.csv'),
                       row.names = 1
    )
}
# select significant genes
hpt18 <- hptTP18[hptTP18[,6] < 0.05,]

In [85]:
# enrichment of selected pathways
if(bool_recomp){
    kkHpt18 <- enrichSelectPath(genes = rownames(hpt18),
                               selectPathway = targetGenes$KEGG,
                               cut_off = 0.05,
                               name = "hpt_TP18",
                               orgdb = org.Rn.eg.db,
                               organism = 'rno'
    )
    write.csv(kkHpt18, 
              file.path(dir_tab, 'kegg_enrichment_hpt_TP18.csv')
    )
} else {
    kkHpt18 = read.csv(file.path(dir_tab, 'kegg_enrichment_hpt_TP18.csv'),
                       row.names = 1,
                       sep = ','
    )
}

In [86]:
# volcano plot differential expressed genes
if(bool_plot){ 
    res = hptTP18[order(hptTP18$padj),]
    results = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                          significant = ifelse(res$padj < 0.05, "FDR<0.05", "Not Sig")
                                   ), row.names = rownames(res)
    )
    vdhpt <- ggplot(results, aes(log2FoldChange, -log10(padj))) +
                    geom_point(aes(col = significant)) +
                    scale_color_manual(values = legv) +
                    geom_text_repel(data = results[1:20,], aes(label = rownames(results[1:20,])))
    ggsave(file.path(dir_fig, "volcano_hpt_TP18.pdf"))
    print(vdhpt)
}

### Hypothalamus TP22

In [87]:
# differntial gene expression
if(bool_recomp){
    hptTP22 <- shrink_dge(dds = dds,
                          method = "deseq2",
                          contrast = c("condition", "hpt22chow", "hpt22hfd")
    )
    hptTP22[is.na(hptTP22$padj),]  <- 1
    write.csv(hptTP22, 
              file.path(dir_tab, 'differential_expression_hpt_TP22.csv')
    )
} else {
    hptTP22 = read.csv(file.path(dir_tab, 'differential_expression_hpt_TP22.csv'),
                       row.names = 1
    )
}
# select significant genes
hpt22 <- hptTP22[hptTP22[,6] < 0.05,]

In [88]:
# enrichment of selected pathways
if(bool_recomp){
    kkHpt22 <- enrichSelectPath(genes = rownames(hpt22),
                               selectPathway = targetGenes$KEGG,
                               cut_off = 0.05,
                               name = "hpt_TP22",
                               orgdb = org.Rn.eg.db,
                               organism = 'rno'
    )
    write.csv(kkHpt22, 
              file.path(dir_tab, 'kegg_enrichment_hpt_TP22.csv')
    )
} else {
    kkHpt22 = read.csv(file.path(dir_tab, 'kegg_enrichment_hpt_TP22.csv'),
                       row.names = 1,
                       sep = ','
    )
}

In [89]:
# volcano plot differential expressed genes
if(bool_plot){ 
    res = hptTP22[order(hptTP22$padj),]
    results = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                          significant = ifelse(res$padj < 0.05, "FDR<0.05", "Not Sig")
                                   ), row.names = rownames(res)
    )
    vdhpt <- ggplot(results, aes(log2FoldChange, -log10(padj))) +
                    geom_point(aes(col = significant)) +
                    scale_color_manual(values = legv) +
                    geom_text_repel(data = results[1:20,], aes(label = rownames(results[1:20,])))
    ggsave(file.path(dir_fig, "volcano_hpt_TP22.pdf"))
    print(vdhpt)
}

In [90]:
# intersection of overlaping genes
if(bool_plot){  
    pdf(file = file.path(dir_fig, "upset_differential_genes_hpt_per_TP.pdf"))
        
    ups <- plotUpset(analyses = list(hptTP2 = hptTP2, 
                                     hptTP6 = hptTP6, 
                                     hptTP10 = hptTP10, 
                                     hptTP14 = hptTP14,
                                     hptTP18 = hptTP18,
                                     hptTP22 = hptTP22
                     ),
                     cut_off = 0.05,
                     col_interest = 6,
                     text.scale = 1.5,
                     matrix.color = "black",
                     main.bar.color = "black",
                     sets.bar.color = "black"
    )
    dev.off()
    ups
}

In [91]:
# bar plot of the number of significant genes per time point in hypothalamus
if(bool_plot){ 
    df <- data.frame(Time = c("TP1_2h", "TP2_6h", "TP3_10h", "TP4_14h", "TP5_18h", "TP6_22h"),
                     Nr_significant_genes = c(nrow(hpt2), nrow(hpt6), nrow(hpt10), nrow(hpt14), nrow(hpt18), nrow(hpt22))
    )
    hpttp <- ggplot(df, aes(x = Time, y = Nr_significant_genes, fill = Time))  +
                    geom_bar(stat = "identity") + 
                    theme_minimal() +
                    theme(legend.position = "none") +
                    scale_fill_manual(values = c(rep('#E6E6E6', 3), rep("#4D4D4D", 3))) 
    
    ggsave(file.path(dir_fig, "barplot_per_timepoint_hpt.pdf"))
    print(hpttp)
}

In [92]:
# Enrichment analysis of selected pathways
if(bool_plot){
    enrich <- ggplot(rbind(kkHpt2, kkHpt6, kkHpt10, kkHpt14, kkHpt18, kkHpt22), 
                     aes(Comparison, Description, col = p.adjust, size = Count)) + 
                     geom_point()+ 
                     scale_colour_gradient(low = "red", high = "blue") +
                     xlab("caricadian genes") + 
                     ylab("enriched pathways")
    
    ggsave(file.path(dir_fig, "kegg_enrichment_selected_pathawys.pdf"))
    print(enrich)
}

In [93]:
# circular visualization of volcano plots - hypothalamus
if(bool_plot){ 
    v_hpt <- data.frame(rbind(cbind(hptTP2, comp = "Hpt_TP2"),
                              cbind(hptTP6, comp = "Hpt_TP6"),
                              cbind(hptTP10, comp = "Hpt_TP10"),
                              cbind(hptTP14, comp = "Hpt_TP14"),
                              cbind(hptTP18, comp = "Hpt_TP18"),
                              cbind(hptTP22, comp = "Hpt_TP22")
                        )
             )
    v_hpt$col <- ifelse(v_hpt$padj > 0.05, "grey", "red")
    v_hpt[,'padj'] <- -log10(v_hpt[,'padj'])
    
    circular_plot(x = v_hpt$log2FoldChange,
                  y = v_hpt$padj, 
                  factors = v_hpt$comp,
                  color = v_hpt$col,
                  leg_name = c('FDR<0.05', 'Not Sig'),
                  leg_col = legv
    )
}

## Target genes over time point

### Hippocampus

In [182]:
if(bool_plot){
    compare_hip <- data.frame()
    for(i in 1:10){
         tg <- targetGenes[[i]]
         c_hip <- rbind(cbind(hippoTP2[rownames(hippoTP2)%in%tg,], factor = "Hippo_TP2"),
                        cbind(hippoTP6[rownames(hippoTP6)%in%tg,], factor = "Hippo_TP6"),
                        cbind(hippoTP10[rownames(hippoTP10)%in%tg,], factor = "Hippo_TP10"),
                        cbind(hippoTP14[rownames(hippoTP14)%in%tg,], factor = "Hippo_TP14"),
                        cbind(hippoTP18[rownames(hippoTP18)%in%tg,], factor = "Hippo_TP18"),
                        cbind(hippoTP22[rownames(hippoTP22)%in%tg,], factor = "Hippo_TP22")
                  )
         c_hip$y <- i                      
         c_hip$name <- names(targetGenes[i])
         c_hip$color <- legcir_colo[i]  
         compare_hip <- rbind(compare_hip, as.data.frame(c_hip)) 
    }
    for(i in 1:nrow(compare_hip)) 
        if(compare_hip[i,'pvalue'] > 0.05) 
            compare_hip[i,'color'] = '#E6E6E6'

    circular_plot(x = compare_hip$log2FoldChange,
                  y = compare_hip$y, 
                  factors = compare_hip$factor,
                  color = compare_hip$color,
                  leg_name = names(targetGenes)[1:10],
                  leg_col = legcir_colo
    )
}

### Hypothalamus

In [183]:
if(bool_plot){
    if(bool_plot){ 
        compare_hpt <- data.frame()
        for(i in 1:10){
            tg <- targetGenes[[i]]
            c_hpt <- rbind(cbind(hptTP2[rownames(hptTP2)%in%tg,], factor = "Hpt_TP2"),
                             cbind(hptTP6[rownames(hptTP6)%in%tg,], factor = "Hpt_TP6"),
                             cbind(hptTP10[rownames(hptTP10)%in%tg,], factor = "Hpt_TP10"),
                             cbind(hptTP14[rownames(hptTP14)%in%tg,], factor = "Hpt_TP14"),
                             cbind(hptTP18[rownames(hptTP18)%in%tg,], factor = "Hpt_TP18"),
                             cbind(hptTP22[rownames(hptTP22)%in%tg,], factor = "Hpt_TP22")     
                       )
            c_hpt$y <- i                      
            c_hpt$name <- names(targetGenes[i])
            c_hpt$color <- legcir_colo[i]
            compare_hpt <- rbind(compare_hpt, as.data.frame(c_hpt))
        }

        for(i in 1:nrow(compare_hpt)) 
            if(compare_hpt[i,'pvalue'] > 0.05)
                compare_hpt[i,'color'] = '#E6E6E6'

        circular_plot(x = compare_hpt$log2FoldChange,
                      y = compare_hpt$y, 
                      factors = compare_hpt$factor,
                      color = compare_hpt$color,
                      leg_name = names(targetGenes)[1:10],
                      leg_col = legcir_colo
        )
    }
}

## Association of whole gene set with periferial data

In [96]:
pfdata <- rbind(pfdat,pfdat)
pfdata <- pfdata[!colnames(dat) %in% prob_samp,]

### Glucose

In [97]:
# association of glucose and hippocampus-chow samples
if(bool_recomp | bool_plot){
    X <- t(vsd[,tissue_diet %in% "hippo_chow"])
    glucose <- pfdata[tissue_diet %in% "hippo_chow",]$Glucose

    glucHipChow <- extract(association(object = gt(glucose, X),
                                       col = c("darkslategray4", "goldenrod4"),
                                       plot = TRUE,
                                       plot_name = "Association between glucose and hippocampus chow data"
                             )      
                     )@result
    write.csv(glucHipChow, 
              file.path(dir_tab, 'association_gluc_hip_chow.csv')
    )
} else {
    glucHipChow = read.csv(file.path(dir_tab, 'association_gluc_hip_chow.csv'),
                           row.names = 1
    )
}

In [98]:
# KEGG enrichment of selected pathawys
if(bool_recomp){
    kkGlucHipChow <- enrichSelectPath(genes = rownames(glucHipChow[(glucHipChow[,1] < 0.05),]),
                                      selectPathway = targetGenes$KEGG,
                                      cut_off = 0.05,
                                      name = "glucHipChow",
                                      orgdb = org.Rn.eg.db,
                                      organism = 'rno'
    )
    write.csv(kkGlucHipChow, 
              file.path(dir_tab, 'kegg_enrichment_gluc_hip_chow.csv')
    )
} else {
    kkGlucHipChow = read.csv(file.path(dir_tab, 'kegg_enrichment_gluc_hip_chow.csv'),
                             row.names = 1,
                             sep = ','
    )
}

In [99]:
# association of glucose and hippocampus-hfd samples
if(bool_recomp | bool_plot){
    X <- t(vsd[,tissue_diet %in% "hippo_hfd"])
    glucose <- pfdata[tissue_diet %in% "hippo_hfd",]$Glucose

    glucHipHfd <- extract(association(object = gt(glucose, X),
                                      col = c("darkslategray4", "goldenrod4"),
                                      plot = TRUE,
                                      plot_name = "Association between glucose and hippocampus hfd data"
                          )      
                  )@result
    write.csv(glucHipHfd,
              file.path(dir_tab, 'association_gluc_hip_hfd.csv')
    )
} else {
    glucHipHfd = read.csv(file.path(dir_tab, 'association_gluc_hip_hfd.csv'),
                          row.names = 1
    )
}

In [100]:
# KEGG enrichment of selected pathawys
if(bool_recomp){
    kkGlucHipHfd <- enrichSelectPath(genes = rownames(glucHipHfd[(glucHipHfd[,1] < 0.05),]),
                                     selectPathway = targetGenes$KEGG,
                                     cut_off = 0.05,
                                     name = "glucHipHfd",
                                     orgdb = org.Rn.eg.db,
                                     organism = 'rno'
    )
    write.csv(kkGlucHipHfd, 
              file.path(dir_tab, 'kegg_enrichment_gluc_hip_hfd.csv')
    )
} else {
    kkGlucHipHfd = read.csv(file.path(dir_tab, 'kegg_enrichment_gluc_hip_hfd.csv'),
                            row.names = 1,
                             sep = ','
    )
}

In [101]:
# association of glucose and hypothalamus-chow samples
if(bool_recomp | bool_plot){
    X <- t(vsd[,tissue_diet %in% "hpt_chow"])
    glucose <- pfdata[tissue_diet %in% "hpt_chow",]$Glucose

    glucHptChow <- extract(association(object = gt(glucose, X),
                                       col = c("darkslategray4","goldenrod4"),
                                       plot = TRUE,
                                       plot_name = "Association between glucose and hypothalamus chow data"
                             )      
                     )@result
    write.csv(glucHptChow, 
              file.path(dir_tab, 'association_gluc_hpt_chow.csv')
    )
} else {
    glucHptChow = read.csv(file.path(dir_tab, 'association_gluc_hpt_chow.csv'),
                           row.names = 1
    )
}    

In [102]:
# KEGG enrichment of selected pathawys
if(bool_recomp){
    kkGlucHptChow <- enrichSelectPath(genes = rownames(glucHptChow[(glucHptChow[,1] < 0.05),]),
                                      selectPathway = targetGenes$KEGG,
                                      cut_off = 0.05,
                                      name = "glucHptChow",
                                      orgdb = org.Rn.eg.db,
                                      organism = 'rno'
    )
    write.csv(kkGlucHptChow, 
              file.path(dir_tab, 'kegg_enrichment_gluc_hpt_chow.csv')
    )
} else {
    kkGlucHptChow = read.csv(file.path(dir_tab, 'kegg_enrichment_gluc_hpt_chow.csv'),
                             row.names = 1,
                             sep = ','
    )
}

In [103]:
# association of glucose and hypothalamus-hfd samples
if(bool_recomp | bool_plot){
    X <- t(vsd[,tissue_diet %in% "hpt_hfd"])
    glucose <- pfdata[tissue_diet %in% "hpt_hfd",]$Glucose

    glucHptHfd <- extract(association(object = gt(glucose, X),
                                      col = c("darkslategray4", "goldenrod4"),
                                      plot = TRUE,
                                      plot_name = "Association between glucose and hypothalamus hfd data"
                          )      
                  )@result
    write.csv(glucHptHfd, 
              file.path(dir_tab, 'association_gluc_hpt_hfd.csv')
    )
} else {
    glucHptHfd = read.csv(file.path(dir_tab, 'association_gluc_hpt_hfd.csv'),
                          row.names = 1
    )
}    

In [104]:
# KEGG enrichment of selected pathawys
if(bool_recomp){
    kkGlucHptHfd <- enrichSelectPath(genes = rownames(glucHptHfd[(glucHptHfd[,1]<0.05),]),
                                      selectPathway = targetGenes$KEGG,
                                      cut_off = 0.05,
                                      name = "glucHptHfd",
                                      orgdb = org.Rn.eg.db,
                                      organism = 'rno'
    )
    write.csv(kkGlucHptHfd, 
              file.path(dir_tab, 'kegg_enrichment_gluc_hpt_chow.csv')
    )
} else {
    kkGlucHptHfd = read.csv(file.path(dir_tab, 'kegg_enrichment_gluc_hpt_chow.csv'),
                            row.names = 1,
                             sep = ','
    )
}

In [106]:
# intersection of overlaping genes
if(bool_plot){  
    pdf(file = file.path(dir_fig, "upset_glucose_association_gene_expression.pdf"))
        
    ups <- plotUpset(analyses = list(glucHipChow = glucHipChow, 
                                     glucHipHfd = glucHipHfd, 
                                     glucHptChow = glucHptChow, 
                                     glucHptHfd = glucHptHfd
                     ),
                     cut_off = 0.05,
                     col_interest = 1
    )
    dev.off()
}
#ups

In [107]:
# Enrichment analysis of selected pathways
if(bool_plot){
    enrich <- ggplot(rbind(kkGlucHipChow, kkGlucHptChow, kkGlucHptHfd), 
                     aes(Comparison, Description, col = p.adjust, size = Count)) + 
                     geom_point()+ 
                     scale_colour_gradient(low = "red", high = "blue") +
                     xlab("caricadian genes") + 
                     ylab("enriched pathways")
    
    ggsave(file.path(dir_fig, "kegg_enrichment_selected_pathawys_glucose_association.pdf"))
    print(enrich)
}

### Insulin

In [108]:
# association of insulin and hippocampus-chow samples
if(bool_recomp | bool_plot){
    X <- t(vsd[,tissue_diet %in% "hippo_chow"])
    insulin <- pfdata[tissue_diet %in% "hippo_chow",]$Insulin

    insulHipChow <- extract(association(object = gt(insulin, X),
                                        col = c("darkslategray4", "goldenrod4"),
                                        plot = TRUE,
                                        plot_name = "Association between insulin and hippocampus chow data"
                             )      
                     )@result
    write.csv(insulHipChow, 
              file.path(dir_tab, 'association_insul_hip_chow.csv')
    )
} else {
    insulHipChow = read.csv(file.path(dir_tab, 'association_insul_hip_chow.csv'),
                            row.names = 1   
    )
}

In [109]:
# KEGG enrichment of selected pathawys
if(bool_recomp){
    kkInsulHipChow <- enrichSelectPath(genes = rownames(insulHipChow[(insulHipChow[,1] < 0.05),]),
                                       selectPathway = targetGenes$KEGG,
                                       cut_off = 0.05,
                                       name = "insulHipChow",
                                       orgdb = org.Rn.eg.db,
                                       organism = 'rno'
    )
    write.csv(kkInsulHipChow, 
              file.path(dir_tab, 'kegg_enrichment_insul_hip_chow.csv')
    )
} else {
    kkInsulHipChow = read.csv(file.path(dir_tab, 'kegg_enrichment_insul_hip_chow.csv'),
                              row.names = 1,
                              sep = ','
    )
}

In [110]:
# association of insulin and hippocampus-hfd samples
if(bool_recomp | bool_plot){
    X <- t(vsd[,tissue_diet %in% "hippo_hfd"])
    insulin <- pfdata[tissue_diet %in% "hippo_hfd",]$Insulin

    insulHipHfd <- extract(association(object = gt(insulin, X),
                                       col = c("darkslategray4", "goldenrod4"),
                                       plot = TRUE,
                                       plot_name = "Association between insulin and hippocampus hfd data"
                          )      
                  )@result
    write.csv(insulHipHfd, 
              file.path(dir_tab, 'association_insul_hip_hfd.csv')
    )
} else {
    insulHipHfd = read.csv(file.path(dir_tab, 'association_insul_hip_hfd.csv'),
                           row.names = 1
    )
}

In [111]:
# KEGG enrichment of selected pathawys
if(bool_recomp){
    kkInsulHipHfd <- enrichSelectPath(genes = rownames(insulHipHfd[(insulHipHfd[,1] < 0.05),]),
                                      selectPathway = targetGenes$KEGG,
                                      cut_off = 0.05,
                                      name = "insulHipHfd",
                                      orgdb = org.Rn.eg.db,
                                      organism = 'rno'
    )
    write.csv(kkInsulHipHfd, 
              file.path(dir_tab, 'kegg_enrichment_insul_hip_hfd.csv')
    )
} else {
    kkInsulHipHfd = read.csv(file.path(dir_tab, 'kegg_enrichment_insul_hip_hfd.csv'),
                             row.names = 1,
                             sep = ','
    )
}

In [112]:
# association of insulin and hypothalamus-chow samples
if(bool_recomp | bool_plot){
    X <- t(vsd[,tissue_diet %in% "hpt_chow"])
    insulin <- pfdata[tissue_diet %in% "hpt_chow",]$Insulin

    insulHptChow <- extract(association(object = gt(insulin, X),
                                        col = c("darkslategray4", "goldenrod4"),
                                        plot = TRUE,
                                        plot_name = "Association between insulin and hypothalamus chow data"
                             )      
                     )@result
    write.csv(insulHptChow, 
              file.path(dir_tab, 'association_insul_hpt_chow.csv')
    )
} else {
    insulHptChow = read.csv(file.path(dir_tab, 'association_insul_hpt_chow.csv'),
                            row.names = 1
    )
}    

In [113]:
# KEGG enrichment of selected pathawys
if(bool_recomp){
    kkInsulHptChow <- enrichSelectPath(genes = rownames(insulHptChow[(insulHptChow[,1] < 0.05),]),
                                       selectPathway = targetGenes$KEGG,
                                       cut_off = 0.05,
                                       name = "insulHptChow",
                                       orgdb = org.Rn.eg.db,
                                       organism = 'rno'
    )
    write.csv(kkInsulHptChow, 
              file.path(dir_tab, 'kegg_enrichment_insul_hpt_chow.csv')
    )
} else {
    kkInsulHptChow = read.csv(file.path(dir_tab, 'kegg_enrichment_insul_hpt_chow.csv'), 
                              row.names = 1,
                              sep = ','
    )
}

In [114]:
# association of insulin and hypothalamus-hfd samples
if(bool_recomp | bool_plot){
    X <- t(vsd[,tissue_diet %in% "hpt_hfd"])
    insulin <- pfdata[tissue_diet %in% "hpt_hfd",]$Insulin

    insulHptHfd <- extract(association(object = gt(insulin, X),
                                       col = c("darkslategray4", "goldenrod4"),
                                       plot = TRUE,
                                       plot_name = "Association between insulin and hypothalamus hfd data"
                          )      
                  )@result
    write.csv(insulHptHfd, 
              file.path(dir_tab, 'association_insul_hpt_hfd.csv')
    )
} else {
    insulHptHfd = read.csv(file.path(dir_tab, 'association_insul_hpt_hfd.csv'),
                           row.names = 1
    )
}    

In [115]:
# KEGG enrichment of selected pathawys
if(bool_recomp){
    kkInsulHptHfd <- enrichSelectPath(genes = rownames(insulHptHfd[(insulHptHfd[,1] < 0.05),]),
                                      selectPathway = targetGenes$KEGG,
                                      cut_off = 0.05,
                                      name = "insulHptHfd",
                                      orgdb = org.Rn.eg.db,
                                      organism = 'rno'
    )
    write.csv(kkInsulHptHfd,
              file.path(dir_tab, 'kegg_enrichment_insul_hpt_chow.csv')
    )
} else {
    kkInsulHptHfd = read.csv(file.path(dir_tab, 'kegg_enrichment_insul_hpt_chow.csv'),
                             row.names = 1,
                             sep = ','
    )
}

In [116]:
# intersection of overlaping genes
if(bool_plot){  
    pdf(file = file.path(dir_fig, "upset_insulin_association_gene_expression.pdf"))
    
    ups <- plotUpset(analyses = list(insulHipChow = insulHipChow, 
                                     insulHipHfd = insulHipHfd, 
                                     insulHptChow = insulHptChow, 
                                     insulHptHfd = insulHptHfd
                     ),
                     cut_off = 0.05,
                     col_interest = 1
    )
    dev.off()
}
ups

ERROR: Error in eval(expr, envir, enclos): объект 'ups' не найден


### Free fatty acids

In [117]:
# association of free fatty acids and hippocampus-chow samples
if(bool_recomp | bool_plot){
    X <- t(vsd[,tissue_diet %in% "hippo_chow"])
    free_fatty_acids <- pfdata[tissue_diet %in% "hippo_chow",]$Free_fatty_acids

    ffaHipChow <- extract(association(object = gt(free_fatty_acids, X),
                                      col = c("darkslategray4", "goldenrod4"),
                                      plot = TRUE,
                                      plot_name = "Association between free fatty acids and hippocampus chow data"
                             )      
                     )@result
    write.csv(ffaHipChow, 
              file.path(dir_tab, 'association_ffa_hip_chow.csv')
    )
} else {
    ffaHipChow = read.csv(file.path(dir_tab, 'association_ffa_hip_chow.csv'), 
                          row.names = 1
    )
}

In [118]:
# KEGG enrichment of selected pathawys
if(bool_recomp){
    kkFfaHipChow <- enrichSelectPath(genes = rownames(ffaHipChow[(ffaHipChow[,1] < 0.05),]),
                                     selectPathway = targetGenes$KEGG,
                                     cut_off = 0.05,
                                     name = "ffaHipChow",
                                     orgdb = org.Rn.eg.db,
                                     organism = 'rno'
    )
    write.csv(kkFfaHipChow, 
              file.path(dir_tab, 'kegg_enrichment_ffa_hip_chow.csv')
    )
} else {
    kkFfaHipChow = read.csv(file.path(dir_tab, 'kegg_enrichment_ffa_hip_chow.csv'),
                            row.names = 1,
                            sep = ','
    )
}

In [119]:
# association of free fatty acids and hippocampus-hfd samples
if(bool_recomp | bool_plot){
    X <- t(vsd[,tissue_diet %in% "hippo_hfd"])
    free_fatty_acids <- pfdata[tissue_diet %in% "hippo_hfd",]$Free_fatty_acids

    ffaHipHfd <- extract(association(object = gt(free_fatty_acids, X),
                                     col = c("darkslategray4", "goldenrod4"),
                                     plot = TRUE,
                                     plot_name = "Association between free fatty acids and hippocampus hfd data"
                          )      
                  )@result
    write.csv(ffaHipHfd, 
              file.path(dir_tab, 'association_ffa_hip_hfd.csv')
    )
} else {
    ffaHipHfd = read.csv(file.path(dir_tab, 'association_ffa_hip_hfd.csv'), 
                         row.names = 1
    )
}

In [120]:
# KEGG enrichment of selected pathawys
if(bool_recomp){
    kkFfaHipHfd <- enrichSelectPath(genes = rownames(ffaHipHfd[(ffaHipHfd[,1]<0.05),]),
                                    selectPathway = targetGenes$KEGG,
                                    cut_off = 0.05,
                                    name = "ffaHipHfd",
                                    orgdb = org.Rn.eg.db,
                                    organism = 'rno'
    )
    write.csv(kkFfaHipHfd, 
              file.path(dir_tab, 'kegg_enrichment_ffa_hip_hfd.csv')
    )
} else {
    kkFfaHipHfd = read.csv(file.path(dir_tab, 'kegg_enrichment_ffa_hip_hfd.csv'), 
                           row.names = 1,
                           sep = ','
    )
}

In [121]:
# association of free fatty acids and hypothalamus-chow samples
if(bool_recomp | bool_plot){
    X <- t(vsd[,tissue_diet %in% "hpt_chow"])
    free_fatty_acids <- pfdata[tissue_diet %in% "hpt_chow",]$Free_fatty_acids

    ffaHptChow <- extract(association(object = gt(free_fatty_acids, X),
                                      col = c("darkslategray4", "goldenrod4"),
                                      plot = TRUE,
                                      plot_name = "Association between free fatty acids and hypothalamus chow data"
                             )      
                     )@result
    write.csv(ffaHptChow, 
              file.path(dir_tab, 'association_ffa_hpt_chow.csv')
    )
} else {
    ffaHptChow = read.csv(file.path(dir_tab, 'association_ffa_hpt_chow.csv'),
                          row.names = 1
    )
}    

In [122]:
# KEGG enrichment of selected pathawys
if(bool_recomp){
    kkFfaHptChow <- enrichSelectPath(genes = rownames(ffaHptChow[(ffaHptChow[,1] < 0.05),]),
                                     selectPathway = targetGenes$KEGG,
                                     cut_off = 0.05,
                                     name = "ffaHptChow",
                                     orgdb = org.Rn.eg.db,
                                     organism = 'rno'
    )
    write.csv(kkFfaHptChow, 
              file.path(dir_tab, 'kegg_enrichment_ffa_hpt_chow.csv')
    )
} else {
    kkFfaHptChow = read.csv(file.path(dir_tab, 'kegg_enrichment_ffa_hpt_chow.csv'),
                            row.names = 1,
                            sep = ','
    )
}

In [123]:
# association of free fatty acids and hippocampus-hfd samples
if(bool_recomp | bool_plot){
    X <- t(vsd[,tissue_diet %in% "hpt_hfd"])
    free_fatty_acids <- pfdata[tissue_diet %in% "hpt_hfd",]$Free_fatty_acids

    ffaHptHfd <- extract(association(object = gt(free_fatty_acids, X),
                                     col = c("darkslategray4", "goldenrod4"),
                                     plot = TRUE,
                                     plot_name = "Association between free fatty acids and hypothalamus chow data"
                          )      
                  )@result
    write.csv(ffaHptHfd, 
              file.path(dir_tab, 'association_ffa_hpt_hfd.csv')
    )
} else {
    ffaHptHfd = read.csv(file.path(dir_tab, 'association_ffa_hpt_hfd.csv'),
                         row.names = 1
    )
}    

In [124]:
# KEGG enrichment of selected pathawys
if(bool_recomp){
    kkFfaHptHfd <- enrichSelectPath(genes = rownames(ffaHptHfd[(ffaHptHfd[,1] < 0.05),]),
                                    selectPathway = targetGenes$KEGG,
                                    cut_off = 0.05,
                                    name = "ffaHptHfd",
                                    orgdb = org.Rn.eg.db,
                                    organism = 'rno'
    )
    write.csv(kkFfaHptHfd, 
              file.path(dir_tab, 'kegg_enrichment_ffa_hpt_chow.csv')
    )
} else {
    kkFfaHptHfd = read.csv(file.path(dir_tab, 'kegg_enrichment_ffa_hpt_chow.csv'),
                           row.names = 1,
                           sep = ','
    )
}

In [125]:
# intersection of overlaping genes
if(bool_plot){  
    pdf(file = file.path(dir_fig, "upset_ffa_association_gene_expression.pdf"))
    ups <- plotUpset(analyses = list(ffaHipChow = ffaHipChow, 
                                     ffaHipHfd = ffaHipHfd, 
                                     ffaHptChow = ffaHptChow, 
                                     ffaHptHfd = ffaHptHfd
                     ),
                     cut_off = 0.05,
                     col_interest = 1
    )
    dev.off()
}
ups

ERROR: Error in eval(expr, envir, enclos): объект 'ups' не найден


In [126]:
# Enrichment analysis of selected pathways
if(bool_plot){
    enrich <- ggplot(rbind(kkFfaHipChow, kkFfaHipHfd, kkFfaHptChow, kkFfaHptHfd), 
                     aes(Comparison, Description, col = p.adjust, size = Count)) + 
                     geom_point()+ 
                     scale_colour_gradient(low = "red", high = "blue") +
                     xlab("caricadian genes") + 
                     ylab("enriched pathways")
    
    ggsave(file.path(dir_fig, "kegg_enrichment_selected_pathawys_ffa_association.pdf"))
    print(enrich)
}

## Association of selected pathways with periferial data

In [127]:
# get genes from the pathways of interest
keggGenes <- list()
for(i in 1:30){
    kg <- pull(read_excel('kegg_genes.xlsx')[,i])
    keggGenes[[targetGenes$KEGG[i]]] <- kg[!is.na(kg)]
}

### Glucose

In [128]:
# association of glucose and hippocampus-chow samples
if(bool_recomp | bool_plot){
    gluc_hip_chow <- numeric()
    r_gluc_hip_chow <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hippo_chow"])
    glucose <- pfdata[tissue_diet %in% "hippo_chow",]$Glucose

    r_gluc_hip_chow <- pathway_association(x_data = X,
                                           y_data = glucose,
                                           pathways = targetGenes$Pathways,
                                           pathways_genes = keggGenes
    )
    write.csv(r_gluc_hip_chow, 
              file.path(dir_tab, 'signif_pathway_association_gluc_hip_chow.csv')
    )
} else {
    r_gluc_hip_chow = read.csv(file.path(dir_tab, 'signif_pathway_association_gluc_hip_chow.csv'), 
                               row.names = 1
    )
}

In [129]:
# association of glucose and hippocampus-hfd samples
if(bool_recomp | bool_plot){
    gluc_hip_hfd <- numeric()
    r_gluc_hip_hfd <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hippo_hfd"])
    glucose <- pfdata[tissue_diet %in% "hippo_hfd",]$Glucose

    r_gluc_hip_hfd <- pathway_association(x_data = X,
                                           y_data = glucose,
                                           pathways = targetGenes$Pathways,
                                           pathways_genes = keggGenes
    )
    write.csv(r_gluc_hip_hfd, 
              file.path(dir_tab, 'signif_pathway_association_gluc_hip_hfd.csv')
    )
} else {
    r_gluc_hip_hfd = read.csv(file.path(dir_tab, 'signif_pathway_association_gluc_hip_hfd.csv'), 
                              row.names = 1
    )
}

ERROR: Error in read.table(file = file, header = header, sep = sep, quote = quote, : first five rows are empty: giving up


In [130]:
# association of glucose and hypothalamus-chow samples
if(bool_recomp | bool_plot){
    gluc_hpt_chow <- numeric()
    r_gluc_hpt_chow <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hpt_chow"])
    glucose <- pfdata[tissue_diet %in% "hpt_chow",]$Glucose

    r_gluc_hpt_chow <- pathway_association(x_data = X,
                                           y_data = glucose,
                                           pathways = targetGenes$Pathways,
                                           pathways_genes = keggGenes
    )
    write.csv(r_gluc_hpt_chow, 
              file.path(dir_tab, 'signif_pathway_association_gluc_hpt_chow.csv')
    )
} else {
    r_gluc_hpt_chow = read.csv(file.path(dir_tab, 'signif_pathway_association_gluc_hpt_chow.csv'), 
                               row.names = 1
    )
}

In [131]:
# association of glucose and hypothalamus-hfd samples
if(bool_recomp | bool_plot){
    gluc_hpt_hfd <- numeric()
    r_gluc_hpt_hfd <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hpt_hfd"])
    glucose <- pfdata[tissue_diet %in% "hpt_hfd",]$Glucose

    r_gluc_hpt_hfd <- pathway_association(x_data = X,
                                          y_data = glucose,
                                          pathways = targetGenes$Pathways,
                                          pathways_genes = keggGenes
    )
    write.csv(r_gluc_hpt_hfd,
              file.path(dir_tab, 'signif_pathway_association_gluc_hpt_hfd.csv')
    )
} else {
    r_gluc_hpt_hfd = read.csv(file.path(dir_tab, 'signif_pathway_association_gluc_hpt_hfd.csv'),
                              row.names = 1
    )
}

### Insulin

In [142]:
# association of insulin and hippocampus-chow samples
if(bool_recomp | bool_plot){
    insul_hip_chow <- numeric()
    r_insul_hip_chow <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hippo_chow"])
    insulin <- pfdata[tissue_diet %in% "hippo_chow",]$Insulin

    r_insul_hip_chow <- pathway_association(x_data = X,
                                            y_data = insulin,
                                            pathways = targetGenes$Pathways,
                                            pathways_genes = keggGenes
    )
    write.csv(r_insul_hip_chow, 
              file.path(dir_tab, 'signif_pathway_association_insul_hip_chow.csv')
    )
} else {
    r_insul_hip_chow = read.csv(file.path(dir_tab, 'signif_pathway_association_insul_hip_chow.csv'),
                                row.names = 1
    )
}

In [143]:
# association of insulin and hippocampus-hfd samples
if(bool_recomp | bool_plot){
    insul_hip_hfd <- numeric()
    r_insul_hip_hfd <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hippo_hfd"])
    insulin <- pfdata[tissue_diet %in% "hippo_hfd",]$Insulin

    r_insul_hip_hfd <- pathway_association(x_data = X,
                                            y_data = insulin,
                                            pathways = targetGenes$Pathways,
                                            pathways_genes = keggGenes
    )
    write.csv(r_insul_hip_hfd, 
              file.path(dir_tab, 'signif_pathway_association_insul_hip_hfd.csv')
    )
} else {
    r_insul_hip_hfd = read.csv(file.path(dir_tab, 'signif_pathway_association_insul_hip_hfd.csv'),
                               row.names = 1
    )
}

In [144]:
# association of insulin and hypothalamus-chow samples
if(bool_recomp | bool_plot){
    insul_hpt_chow <- numeric()
    r_insul_hpt_chow <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hpt_chow"])
    insulin <- pfdata[tissue_diet %in% "hpt_chow",]$Insulin

    r_insul_hpt_chow <- pathway_association(x_data = X,
                                            y_data = insulin,
                                            pathways = targetGenes$Pathways,
                                            pathways_genes = keggGenes
    )
    write.csv(r_insul_hpt_chow, 
              file.path(dir_tab, 'signif_pathway_association_insul_hpt_chow.csv')
    )
} else {
    r_insul_hpt_chow = read.csv(file.path(dir_tab, 'signif_pathway_association_insul_hpt_chow.csv'), 
                                row.names = 1
    )
}

In [148]:
# association of insulin and hypothalamus-hfd samples
if(bool_recomp | bool_plot){
    insul_hpt_hfd <- numeric()
    r_insul_hpt_hfd <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hpt_hfd"])
    insulin <- pfdata[tissue_diet %in% "hpt_hfd",]$Insulin

    r_insul_hpt_hfd <- pathway_association(x_data = X,
                                           y_data = insulin,
                                           pathways = targetGenes$Pathways,
                                           pathways_genes = keggGenes
    )
    write.csv(r_insul_hpt_hfd, 
              file.path(dir_tab, 'signif_pathway_association_insul_hpt_hfd.csv')
    )
} else {
    r_insul_hpt_hfd = read.csv(file.path(dir_tab, 'signif_pathway_association_insul_hpt_hfd.csv'), 
                               row.names = 1
    )
}

### Free fatty acids

In [153]:
# association of free fatty acids and hippocampus-chow samples
if(bool_recomp | bool_plot){
    ffa_hip_chow <- numeric()
    r_ffa_hip_chow <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hippo_chow"])
    free_fatty_acids <- pfdata[tissue_diet %in% "hippo_chow",]$Free_fatty_acids

    r_ffa_hip_chow <- pathway_association(x_data = X,
                                           y_data = free_fatty_acids,
                                           pathways = targetGenes$Pathways,
                                           pathways_genes = keggGenes
    )
    write.csv(r_ffa_hip_chow, 
              file.path(dir_tab, 'signif_pathway_association_ffa_hip_chow.csv')
    )
} else {
    r_ffa_hip_chow = read.csv(file.path(dir_tab, 'signif_pathway_association_ffa_hip_chow.csv'),
                              row.names = 1
    )
}

In [154]:
# association of free fatty acids and hippocampus-hfd samples
if(bool_recomp | bool_plot){
    ffa_hip_hfd <- numeric()
    r_ffa_hip_hfd <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hippo_hfd"])
    free_fatty_acids <- pfdata[tissue_diet %in% "hippo_hfd",]$Free_fatty_acids

    r_ffa_hip_hfd <- pathway_association(x_data = X,
                                         y_data = free_fatty_acids,
                                         pathways = targetGenes$Pathways,
                                         pathways_genes = keggGenes
    )
    write.csv(r_ffa_hip_hfd,
              file.path(dir_tab, 'signif_pathway_association_insul_hip_hfd.csv')
    )
} else {
    r_ffa_hip_hfd = read.csv(file.path(dir_tab, 'signif_pathway_association_ffa_hip_hfd.csv'),
                             row.names = 1
    )
}

In [151]:
# association of free fatty acids and hypothalamus-chow samples
if(bool_recomp | bool_plot){
    ffa_hpt_chow <- numeric()
    r_ffa_hpt_chow <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hpt_chow"])
    free_fatty_acids <- pfdata[tissue_diet %in% "hpt_chow",]$Free_fatty_acids

    r_ffa_hpt_chow <- pathway_association(x_data = X,
                                         y_data = free_fatty_acids,
                                         pathways = targetGenes$Pathways,
                                         pathways_genes = keggGenes
    )
    write.csv(r_ffa_hpt_chow, 
              file.path(dir_tab, 'signif_pathway_association_ffa_hpt_chow.csv')
    )
} else {
    r_ffa_hpt_chow = read.csv(file.path(dir_tab, 'signif_pathway_association_ffa_hpt_chow.csv'),
                              row.names = 1
    )
}

In [155]:
# association of free fatty acids and hypothalamus-hfd samples
if(bool_recomp | bool_plot){
    ffa_hpt_hfd <- numeric()
    r_ffa_hpt_hfd <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hpt_hfd"])
    free_fatty_acids <- pfdata[tissue_diet %in% "hpt_hfd",]$Free_fatty_acids

    r_ffa_hpt_hfd <- pathway_association(x_data = X,
                                         y_data = free_fatty_acids,
                                         pathways = targetGenes$Pathways,
                                         pathways_genes = keggGenes
    )
    write.csv(r_ffa_hpt_hfd,
              file.path(dir_tab, 'signif_pathway_association_ffa_hpt_hfd.csv')
    )
} else {
    r_ffa_hpt_hfd = read.csv(file.path(dir_tab, 'signif_pathway_association_ffa_hpt_hfd.csv'), 
                             row.names = 1
    )
}

## Association of target gene groups with periferial data

### Glucose

In [156]:
# association of glucose and hippocampus-chow samples - target genes
if(bool_recomp | bool_plot){
    gluc_hip_chow <- numeric()
    r_gluc_hip_chow <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hippo_chow"])
    glucose <- pfdata[tissue_diet %in% "hippo_chow",]$Glucose

    r_gluc_hip_chow <- pathway_association(x_data = X,
                                           y_data = glucose,
                                           pathways = names(targetGenes[1:10]),
                                           pathways_genes = targetGenes[1:10]
    )
    write.csv(r_gluc_hip_chow, 
              file.path(dir_tab, 'signif_pathway_association_gluc_hip_chow.csv')
    )
} else {
    r_gluc_hip_chow = read.csv(file.path(dir_tab, 'signif_pathway_association_gluc_hip_chow.csv'), 
                               row.names = 1
    )
}

In [169]:
# association of glucose and hippocampus-hfd samples
if(bool_recomp | bool_plot){
    gluc_hip_hfd <- numeric()
    r_gluc_hip_hfd <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hippo_hfd"])
    glucose <- pfdata[tissue_diet %in% "hippo_hfd",]$Glucose

    r_gluc_hip_hfd <- pathway_association(x_data = X,
                                          y_data = glucose,
                                          pathways = names(targetGenes[1:10]),
                                          pathways_genes = targetGenes[1:10]
    )
    write.csv(r_gluc_hip_hfd, 
              file.path(dir_tab, 'signif_pathway_association_gluc_hip_hfd.csv')
    )
} else {
    r_gluc_hip_hfd = read.csv(file.path(dir_tab, 'signif_pathway_association_gluc_hip_hfd.csv'), 
                              row.names = 1
    )
}

In [158]:
options(repr.plot.width = 7, repr.plot.height = 7)

In [159]:
# association of glucose and hypothalamus-chow samples
if(bool_recomp | bool_plot){
    gluc_hpt_chow <- numeric()
    r_gluc_hpt_chow <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hpt_chow"])
    glucose <- pfdata[tissue_diet %in% "hpt_chow",]$Glucose

    r_gluc_hpt_chow <- pathway_association(x_data = X,
                                           y_data = glucose,
                                           pathways = names(targetGenes[1:10]),
                                           pathways_genes = targetGenes[1:10]
    )
    write.csv(r_gluc_hpt_chow, 
              file.path(dir_tab, 'signif_pathway_association_gluc_hpt_chow.csv')
    )
} else {
    r_gluc_hpt_chow = read.csv(file.path(dir_tab, 'signif_pathway_association_gluc_hpt_chow.csv'), 
                               row.names = 1
    )
}

In [160]:
# association of glucose and hypothalamus-hfd samples
if(bool_recomp | bool_plot){
    gluc_hpt_hfd <- numeric()
    r_gluc_hpt_hfd <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hpt_hfd"])
    glucose <- pfdata[tissue_diet %in% "hpt_hfd",]$Glucose

    r_gluc_hpt_hfd <- pathway_association(x_data = X,
                                          y_data = glucose,
                                          pathways = names(targetGenes[1:10]),
                                          pathways_genes = targetGenes[1:10]
    )
    write.csv(r_gluc_hpt_hfd,
              file.path(dir_tab, 'signif_pathway_association_gluc_hpt_hfd.csv')
    )
} else {
    r_gluc_hpt_hfd = read.csv(file.path(dir_tab, 'signif_pathway_association_gluc_hpt_hfd.csv'),
                              row.names = 1
    )
}

### Insulin

In [170]:
# association of insulin and hippocampus-chow samples
if(bool_recomp | bool_plot){
    insul_hip_chow <- numeric()
    r_insul_hip_chow <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hippo_chow"])
    insulin <- pfdata[tissue_diet %in% "hippo_chow",]$Insulin

    r_insul_hip_chow <- pathway_association(x_data = X,
                                            y_data = insulin,
                                            pathways = names(targetGenes[1:10]),
                                            pathways_genes = targetGenes[1:10]
    )
    write.csv(r_insul_hip_chow, 
              file.path(dir_tab, 'signif_pathway_association_insul_hip_chow.csv')
    )
} else {
    r_insul_hip_chow = read.csv(file.path(dir_tab, 'signif_pathway_association_insul_hip_chow.csv'),
                                row.names = 1
    )
}

In [162]:
# association of insulin and hippocampus-hfd samples
if(bool_recomp | bool_plot){
    insul_hip_hfd <- numeric()
    r_insul_hip_hfd <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hippo_hfd"])
    insulin <- pfdata[tissue_diet %in% "hippo_hfd",]$Insulin

    r_insul_hip_hfd <- pathway_association(x_data = X,
                                            y_data = insulin,
                                            pathways = targetGenes$Pathways,
                                            pathways_genes = keggGenes
    )
    write.csv(r_insul_hip_hfd, 
              file.path(dir_tab, 'signif_pathway_association_insul_hip_hfd.csv')
    )
} else {
    r_insul_hip_hfd = read.csv(file.path(dir_tab, 'signif_pathway_association_insul_hip_hfd.csv'),
                               row.names = 1
    )
}

In [171]:
# association of insulin and hypothalamus-chow samples
if(bool_recomp | bool_plot){
    insul_hpt_chow <- numeric()
    r_insul_hpt_chow <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hpt_chow"])
    insulin <- pfdata[tissue_diet %in% "hpt_chow",]$Insulin

    r_insul_hpt_chow <- pathway_association(x_data = X,
                                            y_data = insulin,
                                            pathways = names(targetGenes[1:10]),
                                            pathways_genes = targetGenes[1:10]
    )
    write.csv(r_insul_hpt_chow, 
              file.path(dir_tab, 'signif_pathway_association_insul_hpt_chow.csv')
    )
} else {
    r_insul_hpt_chow = read.csv(file.path(dir_tab, 'signif_pathway_association_insul_hpt_chow.csv'), 
                                row.names = 1
    )
}

In [172]:
# association of insulin and hypothalamus-hfd samples
if(bool_recomp | bool_plot){
    insul_hpt_hfd <- numeric()
    r_insul_hpt_hfd <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hpt_hfd"])
    insulin <- pfdata[tissue_diet %in% "hpt_hfd",]$Insulin

    r_insul_hpt_hfd <- pathway_association(x_data = X,
                                           y_data = insulin,
                                           pathways = names(targetGenes[1:10]),
                                           pathways_genes = targetGenes[1:10]
    )
    write.csv(r_insul_hpt_hfd, 
              file.path(dir_tab, 'signif_pathway_association_insul_hpt_hfd.csv')
    )
} else {
    r_insul_hpt_hfd = read.csv(file.path(dir_tab, 'signif_pathway_association_insul_hpt_hfd.csv'), 
                               row.names = 1
    )
}

### Free fatty acids

In [173]:
# association of free fatty acids and hippocampus-chow samples
if(bool_recomp | bool_plot){
    ffa_hip_chow <- numeric()
    r_ffa_hip_chow <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hippo_chow"])
    free_fatty_acids <- pfdata[tissue_diet %in% "hippo_chow",]$Free_fatty_acids

    r_ffa_hip_chow <- pathway_association(x_data = X,
                                           y_data = free_fatty_acids,
                                           pathways = names(targetGenes[1:10]),
                                           pathways_genes = targetGenes[1:10]
    )
    write.csv(r_ffa_hip_chow, 
              file.path(dir_tab, 'signif_pathway_association_ffa_hip_chow.csv')
    )
} else {
    r_ffa_hip_chow = read.csv(file.path(dir_tab, 'signif_pathway_association_ffa_hip_chow.csv'),
                              row.names = 1
    )
}

In [174]:
# association of free fatty acids and hippocampus-hfd samples
if(bool_recomp | bool_plot){
    ffa_hip_hfd <- numeric()
    r_ffa_hip_hfd <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hippo_hfd"])
    free_fatty_acids <- pfdata[tissue_diet %in% "hippo_hfd",]$Free_fatty_acids

    r_ffa_hip_hfd <- pathway_association(x_data = X,
                                         y_data = free_fatty_acids,
                                         pathways = names(targetGenes[1:10]),
                                         pathways_genes = targetGenes[1:10]
    )
    write.csv(r_ffa_hip_hfd,
              file.path(dir_tab, 'signif_pathway_association_insul_hip_hfd.csv')
    )
} else {
    r_ffa_hip_hfd = read.csv(file.path(dir_tab, 'signif_pathway_association_ffa_hip_hfd.csv'),
                             row.names = 1
    )
}

In [167]:
# association of free fatty acids and hypothalamus-chow samples
if(bool_recomp | bool_plot){
    ffa_hpt_chow <- numeric()
    r_ffa_hpt_chow <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hpt_chow"])
    free_fatty_acids <- pfdata[tissue_diet %in% "hpt_chow",]$Free_fatty_acids

    r_ffa_hpt_chow <- pathway_association(x_data = X,
                                         y_data = free_fatty_acids,
                                         pathways = names(targetGenes[1:10]),
                                         pathways_genes = targetGenes[1:10]
    )
    write.csv(r_ffa_hpt_chow, 
              file.path(dir_tab, 'signif_pathway_association_ffa_hpt_chow.csv')
    )
} else {
    r_ffa_hpt_chow = read.csv(file.path(dir_tab, 'signif_pathway_association_ffa_hpt_chow.csv'),
                              row.names = 1
    )
}

In [175]:
# association of free fatty acids and hypothalamus-hfd samples
if(bool_recomp | bool_plot){
    ffa_hpt_hfd <- numeric()
    r_ffa_hpt_hfd <- data.frame()
    X <- t(vsd[,tissue_diet %in% "hpt_hfd"])
    free_fatty_acids <- pfdata[tissue_diet %in% "hpt_hfd",]$Free_fatty_acids

    r_ffa_hpt_hfd <- pathway_association(x_data = X,
                                         y_data = free_fatty_acids,
                                         pathways = names(targetGenes[1:10]),
                                         pathways_genes = targetGenes[1:10]
    )
    write.csv(r_ffa_hpt_hfd,
              file.path(dir_tab, 'signif_pathway_association_ffa_hpt_hfd.csv')
    )
} else {
    r_ffa_hpt_hfd = read.csv(file.path(dir_tab, 'signif_pathway_association_ffa_hpt_hfd.csv'), 
                             row.names = 1
    )
}

### Reconstrucction of temporal gene-gene interactions

In [421]:
make_uniq <- function (xs, sep = "-"){
    cumcount <- function (xs) {
        counts <- new.env(parent = emptyenv())
        setNames(vapply(xs,
                        function(x) counts[[x]] <- 1L + mget(x,counts, ifnotfound = 0L)[[1]], 
                        integer(1)), 
                        xs
        )
    }
    xs[] <- paste(xs, cumcount(xs), sep = sep)
    return(xs)
}

# time points for hippocampus
timep <- as.factor(rep(c(rep(1, 5), rep(2, 5), rep(3, 5), rep(4, 5), rep(5, 5), rep(6, 5)), 2))

# get the genes from the patway of interest and order the samples
hipp <- dat[rownames(dat) %in% keggGenes[[1]],1:60]
hipp <- hipp[,order(timep)]

# vector with group indices 
id <- c(rep(0,5),rep(1,5))

# modify the col names of data
timep <- timep[order(timep)]
colnames(hipp) <- make_uniq(as.character(timep))

# set to NA problematic samples
hipp[,colnames(dat[,1:60]) %in% prob_samp] <- NA
p <- nrow(hipp) # number of genes

In [422]:
# Gausianization of the data
hipp.npn = huge.npn(hipp)
# Convert a longitudinal object into an array.
Y <- longitudinal2array(t(hipp.npn))
dim(Y)

Conducting the nonparanormal (npn) transformation via shrunkun ECDF....done.


[1] 29  6 10

In [423]:
# automatic penalty parameter selection for fused VAR(1)
optLambdas <- optPenaltyVAR1fused(Y,
                                  id,
                                  rep(10^(-10), 3), 
                                  rep(1000, 3),
                                  optimizer = "nlm"
)

In [424]:
# fused ridge ML estimation of multiple VAR(1) model
VAR1hats <- ridgeVAR1fused(Y, 
                           id,
                           optLambdas[1],
                           optLambdas[2], 
                           optLambdas[3]
)

In [425]:
Achow <- VAR1hat$As[1:p,]
Ahfd <- VAR1hat$As[(p+1):(2*p),]
Phat <- VAR1hat$P
rownames(Achow) <- colnames(Achow) <- rownames(Ahfd) <- colnames(Ahfd) <-
     rownames(Phat) <- colnames(Phat) <- rownames(hipp)

In [176]:
if(bool_plot){
    edgeHeat(Achow)
    edgeHeat(Ahfd)
    edgeHeat(Phat)
}

In [177]:
if(bool_plot){
    Achow1 <- matrix(0, p, p) 
    Achow1[sparsifyVAR1(Achow, solve(Phat), threshold="localFDR", verbose = FALSE)$nonzeros] <- 1 
    edgeHeat(Achow1) 
}

In [178]:
if(bool_plot){
    Ahfd1 <- matrix(0, p, p) 
    Ahfd1[sparsifyVAR1(Ahfd, solve(Phat), threshold="localFDR", verbose = FALSE)$nonzeros] <- 1 
    edgeHeat(Ahfd1) 
}

In [179]:
if(bool_plot){
    Phat1 <- sparsify(Phat, threshold = "localFDR", verbose = FALSE)$sparsePrecision
    edgeHeat(Phat1)
}

In [180]:
if(bool_plot){
    graphVAR1(Achow1, Phat1, type="TSCG",nNames = rownames(hipp), vertex.label.cex=1,vertex.label.color.T0="red", 
              vertex.label.color.T1="red",vertex.size=10)
}

In [181]:
if(bool_plot){
    graphVAR1(Ahfd1, Phat1, type="TSCG",nNames = rownames(hipp), vertex.label.cex=1,vertex.label.color.T0="red", 
              vertex.label.color.T1="red",vertex.size=10)
}